# Crossovers in genetic algorithms







**Crossover**, also known as recombination, is the process of combining two parent solutions to generate new offspring. This results in two offspring, each carrying some genetic information from both parents. The idea is to exchange some parts of the parents' strings to create a new string that may inherit some good traits from both parents, and it is analogous to the crossover that happens during sexual reproduction in biology.

In evolutionary computation, various algorithms may use distinct data structures to store genetic information (see [genetic algorithms](genetic_algorithms.ipynb)). Each genetic representation can be combined with different crossover operators. Bit arrays, vectors of real numbers, or trees are some typical data structures that can be recombined with crossover. Note that the same individual may be selected as both parents, and one individual may take part in producing more than one offspring. 

**Main categories** of crossover operators are:  <br>
(1) <span style="color:#EE8877">Asexual:</span> an offspring is generated from one parent, <br>
(2) <span style="color:#77EFE5">Sexual:</span> two parents produce one or more offspring, and <br>
(3) <span style="color:#8799EE">Multi-recombination:</span> more than two parents are used to produce one or more offspring.

<figure>
    <center> <img src="./pics/Crossover/crossover.png"  alt='missing' width="400"  ><center/>
<figure/>

## **Contents**

<a href='#chrom-def'>01. Chromosome Definition</a>  
 
<a href='#one-point'>02. One-point Crossover</a>

<a href='#multi-point'>03. Multi-point Crossover</a>

<a href='#uniform'>04. Uniform Crossover</a>

<a href='#multi-parent'>05. Multi-parent Crossover</a>

<a href='#order'>06. Order Crossover (OX)</a>

<a href='#partially-mapped'>07. Partially Mapped Crossover (PMX)</a>

<a href='#cycle'>08. Cycle Crossover (CX)</a>

<a href='#intermediate'>09. Heuristic Crossover (HC) or Intermediate Crossover (IC)</a>

<a href='#shuffle'>10. Shuffle Crossover (SX)</a>

<a href='#reduce-surrogate'>11. Reduce Surrogate Crossover (RSX)</a>

<a href='#average'>12. Average Crossover (AX)</a>

<a href='#discrete'>13. Discrete Crossover (DC)</a>

<a href='#flat'>14. Flat Crossover (FC)</a>

<a href='#multivariate'>15. Multivariate Crossover</a>

<a href='#count-preserving'>16. Count Preserving Crossover (CPC)</a>

<a href='#modified-order'>17. Modified Order Crossover (MOC)</a>

<a href='#random-respectful'>18. Random Respectful Crossover (RRC)</a>

<a href='#masked'>19. Masked Crossover</a>

<a href='#1-bit-adaptive'>20. 1-Bit Adaptive Crossover (1BX)</a>

<a href='#modified-partially-mapped'>21. Modified Partially Mapped Crossover (MPMX)</a>

<a href='#order-based'>22. Order-Based Crossover (OBX)</a>

<a href='#position-based'>23. Position-Based Crossover (POS)</a>

<a href='#voting-recombination'>24. Voting Recombination Crossover (VR)</a>

<a href='#maximal-preservation'>25. Maximal Preservation Crossover (MPX)</a>

<a href='#position'>26. Position Crossover (PX)</a>

<a href='#homologous'>27. Homologous Crossover (HX)</a>

<a href='#complete-subtour-exchange'>28. Complete Subtour Exchange Crossover (CSEX)</a>

<a href='#edge-recombination'>29. Edge Recombination Crossover (ER)</a>

<a href='#alternate-edges'>30. Alternate Edges Crossover (AEX)</a>

<a href='#half-uniform'>31. Half Uniform Crossover (HUX)</a>

<a href='#highly-disruptive'>32. Highly Disruptive Crossover (HDX)</a>

<a href='#blend'>33. Blend Crossover (BLX)</a>

<a href='#linear-recombination'>34. Linear Recombination Crossover</a>

<a href='#directional-heuristic'>35. Directional Heuristic Crossover</a>

<a href='#geometrical'>36. Geometrical Crossover</a>

<a href='#simulated-binary'>37. Simulated Binary Crossover (SBX)</a>

<a href='#hill-climbing'>38. Hill-Climbing Crossover</a>

<a href='#arithmetic'>39. Arithmetic Crossover</a>


## <a id='chrom-def'>Chromosome Definition</a>

First, we need to implement the chromosome class, a function to calculate the fitness of individuals (chromosomes) in the population, and some helper functions.

In [1]:
import numpy as np

class Chromosome():
    """
    Description of class `Chromosome`:
    This class represents a simple chromosome. In the method describe, a simple description
    of the chromosome is provided, when it is called. 
    """
    def __init__(self, genes, id_=None, fitness=-1):
        self.id_ = id_
        self.genes = genes
        self.fitness = fitness       
       
    def describe(self): 
        """
        Prints the ID, fitness, and genes
        """
        print(f"ID=#{self.id_}, Fitness={self.fitness}, \nGenes=\n{self.genes}")
 
    def get_chrom_length(self): 
        """
        Returns the length of `self.genes`
        """
        return len(self.genes)

def find_chrom_type(chrom):
    """
    This function, takes a chromosome and returns its type (binary, integer, 
    or a floating-point chromosome).
    chrom: The chromoseme which its type is calculated and returned.
    """ 
    # if a floating-point individual
    if np.issubdtype(chrom.dtype, np.floating):
        return "float"
    # if a binary individual
    elif np.array_equal(chrom, chrom.astype(bool)):
        return "binary"
    # if an integer individual
    elif np.issubdtype(chrom.dtype, np.integer):
        return "integer"
    
    return "binary"

# Note that calculating the fitness of a chromosome depends on the problem at hand,
# and what you see in this function is just for understanding the concept.
def fitness_function(chrom): 
    """
    This function, takes a chromosome and returns a value as its fitness.
    chrom: The chromoseme which its fitness is calculated and returned.
    """  
    chrom_type = find_chrom_type(chrom)
    # if we have a binary chromosome (count the number of 1s)
    if chrom_type == "binary":
        return np.count_nonzero(chrom == 1)
    # if we have an integer chromosome (count values between 0 and 4)
    elif chrom_type == "integer":
        return np.count_nonzero((chrom >= 0) & (chrom <= 4))
    # if we have a floating-point chromosome (count values between 0 and 0.5) 
    elif chrom_type == "float":
        return np.count_nonzero((chrom >= 0) & (chrom <= 0.5))
    
    return -1


def sorted_random_numbers(point_nums, interval):
    """
    Returns several sorted random integers in an interval.
    point_nums: number of points to be selected.
    interval: the interval which the numbers are selected.
    """    
    points = []
    for _ in range(point_nums):
        num = np.random.randint(1, abs(interval))
        while num in points:
            num = np.random.randint(1, abs(interval))
        points.append(num)
    return sorted(points)




## <a id='one-point'>One-point Crossover</a>
In one-point crossover, two chromosomes as parents are selected and two offspring are produced. One point in the parent chromosomes is selected randomly, and the genes of the first part of child-one will be filled by the parent-one, and the second part of the child-one will be filled by the second part of the parent-two. For child-2, the first part is filled by parent-two, and the second part is filled by the parent-one.

<figure>
    <center> <img src="./pics/Crossover/one_point_crossover.png"  alt='missing' width="450"  ><center/>
<figure/>

In [2]:
import numpy as np
from copy import deepcopy

def one_point_crossover(parent_one, parent_two, x_point, pc):
    """
    This function takes two parents, and performs One-point crossover on them. 
    parent_one: First parent
    parent_two: Second parent
    x_point: The crossover point
    pc: The probability of crossover 
    """  
    
    print("\nParents")
    print("=================================================")
    Chromosome.describe(parent_one)
    Chromosome.describe(parent_two)
    

    chrom_length = Chromosome.get_chrom_length(parent_one)
    child_one = Chromosome(genes= np.array([0]*chrom_length),id_=0,fitness = 125.2)  
    child_two = Chromosome(genes= np.array([0]*chrom_length),id_=1,fitness = 125.2)
    point = np.random.randint(0, abs(chrom_length))
    if x_point != 0: point = x_point
    # point = 4
    if np.random.rand() < pc:  # if pc is greater than random number
        for i in range(chrom_length):
            if i<point:
                child_one.genes[i] = parent_one.genes[i]
                child_two.genes[i] = parent_two.genes[i]
            else:
                child_one.genes[i] = parent_two.genes[i]
                child_two.genes[i] = parent_one.genes[i]
    else:  # if pc is less than random number then don't make any change
        child_one = deepcopy(parent_one)
        child_two = deepcopy(parent_two)

    # updating fitnesses for children
    child_one.fitness = fitness_function(child_one.genes)
    child_two.fitness = fitness_function(child_two.genes)

    return child_one, child_two


# parents
chrom0 = np.array([0,0,0,0,0,0,0,0,0,0,0])
chrom1 = np.array([1,1,1,1,1,1,1,1,1,1,1])
parent_one = Chromosome(chrom0, id_= 0, fitness = fitness_function(chrom0))  
parent_two = Chromosome(chrom1, id_= 1, fitness = fitness_function(chrom1))

CROSS = one_point_crossover(parent_one, parent_two, 0, 1)

if __name__ == '__main__':
    print("\nChildren")
    print("=================================================")
    for index, _ in enumerate(CROSS):  #two children
        Chromosome.describe(CROSS[index])




Parents
ID=#0, Fitness=0, 
Genes=
[0 0 0 0 0 0 0 0 0 0 0]
ID=#1, Fitness=11, 
Genes=
[1 1 1 1 1 1 1 1 1 1 1]

Children
ID=#0, Fitness=3, 
Genes=
[0 0 0 0 0 0 0 0 1 1 1]
ID=#1, Fitness=8, 
Genes=
[1 1 1 1 1 1 1 1 0 0 0]


## <a id='multi-point'>Multi-point Crossover</a>

This crossover is an extension of one-point crossover. Several points in the parent chromosomes are randomly selected, and the genes from the parents will be transferred to children in the same order, but the parent who is transferring the genes will change at each point.

<figure>
    <center> <img src="./pics/Crossover/multi_point_crossover.png"  alt='missing' width="450"  ><center/>
<figure/>

In [3]:
import numpy as np
from copy import deepcopy

def multi_point_crossover(parent_one, parent_two, pc):
    """
    This function takes two parents, and performs Multi-point crossover on them by selecting 
    several points randomly, as crossover points. 
    parent_one: First parent
    parent_two: Second parent
    pc: The probability of crossover 
    """  
    
    print("\nParents")
    print("=================================================")
    Chromosome.describe(parent_one)
    Chromosome.describe(parent_two)
    
    chrom_length = Chromosome.get_chrom_length(parent_one)
    child_one = Chromosome(genes= np.array([0]*chrom_length),id_=0,fitness = 125.2)   
    child_two = Chromosome(genes= np.array([0]*chrom_length),id_=1,fitness = 125.2)
    point_nums = np.random.randint(1, abs(chrom_length)) #a random number for the number of points
    points = sorted_random_numbers(point_nums,chrom_length)
    count = 0
    if np.random.rand() < pc:  # if pc is greater than random number
        prev_index=0
        for _ in range(len(points)):
            if count%2==0:
                for i in range(prev_index,chrom_length,1):
                    if i<points[count]:
                        child_one.genes[i] = parent_one.genes[i]
                        child_two.genes[i] = parent_two.genes[i]
                prev_index = points[count]
                count +=1
            elif count%2!=0:
                for i in range(prev_index,chrom_length,1):
                    if i<points[count]:
                        child_one.genes[i] = parent_two.genes[i]
                        child_two.genes[i] = parent_one.genes[i]
                prev_index = points[count]
                count +=1          
        # for the last part
        if count%2==0:
            for i in range(points[len(points)-1],chrom_length,1):         
                child_one.genes[i] = parent_one.genes[i]
                child_two.genes[i] = parent_two.genes[i]
        elif count%2!=0:
            for i in range(points[len(points)-1],chrom_length,1):
                child_one.genes[i] = parent_two.genes[i]
                child_two.genes[i] = parent_one.genes[i]

    else:  # if pc is less than random number then don't make any change
        child_one = deepcopy(parent_one)
        child_two = deepcopy(parent_two)

    # updating fitnesses for children
    child_one.fitness = fitness_function(child_one.genes)
    child_two.fitness = fitness_function(child_two.genes)
    
    return child_one, child_two


# parents
chrom0 = np.array([0,0,0,0,0,0,0,0,0,0,0])
chrom1 = np.array([1,1,1,1,1,1,1,1,1,1,1])
parent_one = Chromosome(chrom0, id_= 0, fitness = fitness_function(chrom0))  
parent_two = Chromosome(chrom1, id_= 1, fitness = fitness_function(chrom1))

CROSS = multi_point_crossover(parent_one, parent_two, 1)

if __name__ == '__main__':
    print("\nChildren")
    print("=================================================")
    for index, _ in enumerate(CROSS):  #two children
        Chromosome.describe(CROSS[index])




Parents
ID=#0, Fitness=0, 
Genes=
[0 0 0 0 0 0 0 0 0 0 0]
ID=#1, Fitness=11, 
Genes=
[1 1 1 1 1 1 1 1 1 1 1]

Children
ID=#0, Fitness=4, 
Genes=
[0 0 0 1 1 0 0 0 1 1 0]
ID=#1, Fitness=7, 
Genes=
[1 1 1 0 0 1 1 1 0 0 1]


## <a id='uniform'>Uniform Crossover</a>

In uniform crossover, a random number between 0 and 1 is produced for each gene of the parent chromosome. If that random number is less than 0.5, then the gene from parent-one will be transferred to child-one; otherwise, the gene will come from parent-two.
The same thing will happen to child two, but the condition is reversed. Which means that whatever gene is transferred to child-one from one of the parents, the gene from the other parent in that position is transferred to child-two.

<figure>
    <center> <img src="./pics/Crossover/uniform_crossover.png"  alt='missing' width="450"  ><center/>
<figure/>

In [4]:
import numpy as np
from copy import deepcopy

def claculate_mask(chrom_size, px):
    """
    This method, performs the mask calculation for the Uniform crossover.
    chrom_size: The mask has the same size of the chromosome.
    px: A control parameter for the number of ones in the mask vector.
    """  
    mask = [0]*chrom_size
    for i in range(chrom_size):
        prob = np.random.rand()
        if prob <= px:
            mask[i]=1
    return np.array(mask)
            
def uniform_crossover(parent_one, parent_two, pc):
    """
    This function takes two parents, and performs Uniform crossover on them. 
    parent_one: First parent
    parent_two: Second parent
    pc: The probability of crossover
    """  
    
    print("\nParents")
    print("=================================================")
    Chromosome.describe(parent_one)
    Chromosome.describe(parent_two)
    
    chrom_length = Chromosome.get_chrom_length(parent_one)
    child_one = Chromosome(genes= np.array([0]*chrom_length),id_=0,fitness = 125.2) 
    child_two = Chromosome(genes= np.array([0]*chrom_length),id_=1,fitness = 125.2)
    if np.random.rand() < pc:  # if pc is greater than random number
        mask = claculate_mask(chrom_length, np.random.uniform(low=0.2, high=0.85))
        print("\n Mask Vector")
        print("=================================================")
        print(mask)
        for i in range(chrom_length):
            if mask[i]==1:
                child_one.genes[i] = parent_one.genes[i]
                child_two.genes[i] = parent_two.genes[i]
            else:
                child_one.genes[i] = parent_two.genes[i]
                child_two.genes[i] = parent_one.genes[i]
           
    else:  # if pc is less than random number then don't make any change
        child_one = deepcopy(parent_one)
        child_two = deepcopy(parent_two)

    # updating fitnesses for children
    child_one.fitness = fitness_function(child_one.genes)
    child_two.fitness = fitness_function(child_two.genes)

    return child_one, child_two

# parents
chrom0 = np.array([0,0,0,0,0,0,0,0,0,0,0])
chrom1 = np.array([1,1,1,1,1,1,1,1,1,1,1])
parent_one = Chromosome(chrom0, id_= 0, fitness = fitness_function(chrom0))  
parent_two = Chromosome(chrom1, id_= 1, fitness = fitness_function(chrom1))
CROSS = uniform_crossover(parent_one, parent_two, 1)

if __name__ == '__main__':
    print("\nChildren")
    print("=================================================")
    for index, _ in enumerate(CROSS):  #two children
        Chromosome.describe(CROSS[index])



Parents
ID=#0, Fitness=0, 
Genes=
[0 0 0 0 0 0 0 0 0 0 0]
ID=#1, Fitness=11, 
Genes=
[1 1 1 1 1 1 1 1 1 1 1]

 Mask Vector
[0 1 0 1 0 0 0 1 0 0 0]

Children
ID=#0, Fitness=8, 
Genes=
[1 0 1 0 1 1 1 0 1 1 1]
ID=#1, Fitness=3, 
Genes=
[0 1 0 1 0 0 0 1 0 0 0]


## <a id='multi-parent'>Multi-parent Crossover</a>

In multi-parent crossover, several parents from the population are selected and one child is produced. Each gene of the offspring is from one of its parents. For instance, if we consider six chromosomes as the parents, the offspring will be something like this.

<figure>
    <center> <img src="./pics/Crossover/multi_parent_crossover.png"  alt='missing' width="450"  ><center/>
<figure/>

In [5]:
import numpy as np
from copy import deepcopy         

def multi_parent_crossover(parents, pc):
    """
    This function takes several parents, and performs Multi-parent crossover on them. 
    In this method, number of parents is equal to the length of the chromosome.
    parent_one: First parent
    parent_two: Second parent
    pc: The probability of crossover
    """  
    
    chrom_length = Chromosome.get_chrom_length(parents[0])
    
    print("\nParents")
    print("=================================================")
    for i in range(chrom_length):
        Chromosome.describe(parents[i])

    child_one = Chromosome(genes= np.array([0]*chrom_length),id_=0,fitness = 125.2)   
    if np.random.rand() < pc:  # if pc is greater than random number
        for i in range (chrom_length):
            child_one.genes[i]=parents[i].genes[i]           
       
    else:  # if pc is less than random number then the children will be one of the parents involved
        child_one = deepcopy(parents[np.random.randint(0, chrom_length)])

    # updating fitnesses for children
    child_one.fitness = fitness_function(child_one.genes)
        
    return child_one

# parents
parents = []
genes0 = np.array([0, 0, 0, 0, 0, 0])
genes1 = np.array([1, 1, 1, 1, 1, 1])
genes2 = np.array([2, 2, 2, 2, 2, 2])
genes3 = np.array([3, 3, 3, 3, 3, 3])
genes4 = np.array([4, 4, 4, 4, 4, 4])
genes5 = np.array([5, 5, 5, 5, 5, 5])

parents.append(Chromosome(genes= np.array(genes0), id_=0, fitness = fitness_function(genes0)))
parents.append(Chromosome(genes= np.array(genes1), id_=1, fitness = fitness_function(genes1)))
parents.append(Chromosome(genes= np.array(genes2), id_=2, fitness = fitness_function(genes2)))
parents.append(Chromosome(genes= np.array(genes3), id_=3, fitness = fitness_function(genes3)))
parents.append(Chromosome(genes= np.array(genes4), id_=4, fitness = fitness_function(genes4)))
parents.append(Chromosome(genes= np.array(genes5), id_=5, fitness = fitness_function(genes5)))


CROSS = multi_parent_crossover(parents, 1)

if __name__ == '__main__':
    print("\nChildren")
    print("=================================================")
    
    Chromosome.describe(CROSS)





Parents
ID=#0, Fitness=0, 
Genes=
[0 0 0 0 0 0]
ID=#1, Fitness=6, 
Genes=
[1 1 1 1 1 1]
ID=#2, Fitness=6, 
Genes=
[2 2 2 2 2 2]
ID=#3, Fitness=6, 
Genes=
[3 3 3 3 3 3]
ID=#4, Fitness=6, 
Genes=
[4 4 4 4 4 4]
ID=#5, Fitness=0, 
Genes=
[5 5 5 5 5 5]

Children
ID=#0, Fitness=5, 
Genes=
[0 1 2 3 4 5]


## <a id='order'>Order Crossover (OX)</a>

In order crossover, two points are selected randomly in the parents. Then the genes between these two points from parent-one is directly transferred to child-one and from parent-two to child-two. After that, for child-one we start from the second point of parent-two and we transfer to child-one what is not already transferred to child-one. We do the same thing for child-two with parent-one.

<figure>
    <center> <img src="./pics/Crossover/order_cross.png"  alt='missing' width="450"  ><center/>
<figure/>

In [6]:
import numpy as np
from copy import deepcopy  

def order_crossover(parent_one, parent_two, pc):
    """
    This function takes two parents, and performs Order crossover on them. 
    parent_one: First parent
    parent_two: Second parent
    pc: The probability of crossover
    """  
    
    print("\nParents")
    print("=================================================")
    Chromosome.describe(parent_one)
    Chromosome.describe(parent_two)
    
    chrom_length = Chromosome.get_chrom_length(parent_one)
    child_one = Chromosome(genes= np.array([-1]*chrom_length),id_=0,fitness = 125.2)  
    child_two = Chromosome(genes= np.array([-1]*chrom_length),id_=1,fitness = 125.2)
    point1 = np.random.randint(1, int(chrom_length/2))
    point2 = np.random.randint(int(chrom_length/2), chrom_length)
    
    trans_elements1 = [] #elements transfered to child1 from parent1
    trans_elements2 = [] #elements transfered to child2 from parent2
    list1 = [] #elements to be transfered to child1 from parent2 
    list2 = [] #elements to be transfered to child2 from parent1

    if np.random.rand() < pc:  # if pc is greater than random number
        for i in range(chrom_length):  # transfer genes
            if i >= point1 and i < point2:
                child_one.genes[i] = parent_one.genes[i]
                child_two.genes[i] = parent_two.genes[i]
                trans_elements1.append(child_one.genes[i])
                trans_elements2.append(child_two.genes[i])
        for i in range(point2,chrom_length):
            list1.append(parent_two.genes[i]) #for c1 from p2
            list2.append(parent_one.genes[i]) #for c2 from p1
        for i in range(point2):
            list1.append(parent_two.genes[i]) 
            list2.append(parent_one.genes[i]) 
        delete1 = [] #index of the elements that should be deleted from list1
        delete2 = [] #index of the elements that should be deleted from list2
        for i in range(chrom_length):
            if list1[i] in trans_elements1: delete1.append(i)
            if list2[i] in trans_elements2: delete2.append(i)
        count = 0
        for i in range(len(delete1)): #deleting the redundant elements
            list1.pop(delete1[i]-count)
            list2.pop(delete2[i]-count)
            count +=1
        count = 0
        for i in range(point2,chrom_length):
            child_one.genes[i] = list1[count]
            child_two.genes[i] = list2[count]
            count +=1 
            
        for i in range(point1):
            child_one.genes[i] = list1[count]
            child_two.genes[i] = list2[count]
            count +=1

    else:  # if pc is less than random number then don't make any change
        child_one = deepcopy(parent_one)
        child_two = deepcopy(parent_two)

    # updating fitnesses for children
    child_one.fitness = fitness_function(child_one.genes)
    child_two.fitness = fitness_function(child_two.genes)
    
    return child_one, child_two



# parents
chrom0 = np.array([0,1,2,3,4,5,6,7,8,9])
chrom1 = np.array([4,5,2,1,8,0,7,6,9,3])
parent_one = Chromosome(chrom0, id_= 0, fitness = fitness_function(chrom0))  
parent_two = Chromosome(chrom1, id_= 1, fitness = fitness_function(chrom1))
CROSS = order_crossover(parent_one, parent_two, 1)

if __name__ == '__main__':
    print("\nChildren")
    print("=================================================")
    for index, _ in enumerate(CROSS):  #two children
        Chromosome.describe(CROSS[index])



Parents
ID=#0, Fitness=5, 
Genes=
[0 1 2 3 4 5 6 7 8 9]
ID=#1, Fitness=5, 
Genes=
[4 5 2 1 8 0 7 6 9 3]

Children
ID=#0, Fitness=5, 
Genes=
[0 7 2 3 4 5 6 9 1 8]
ID=#1, Fitness=5, 
Genes=
[5 6 2 1 8 0 7 9 3 4]


## <a id='partially-mapped'>Partially Mapped Crossover (PMX)</a>

In this crossover, two points are selected randomly in the parents. Then the genes between these two points from parent-one are directly transferred to child-one and from parent-two to child-two. After that, for child-one, we look in parent-two and transfer all the genes that can be transferred (not already exist in the middle section of child-one), directly to child-one. And we do the same thing for child-two but with parent-one.

For the last step, for child-one, we go through the middle section of parent-two and find an element that does not already exist in child-one. Then, we search the value that exists in its corresponding index in child-one in parent-two. If the corresponding index of the searched value in parent-two is empty in child-one, we put the element in that spot. Else, we keep trying until we find an empty spot. 

In the following example, after the first and the second step, we have these for child-one and child-two:

`child-one = [0 1 -1 3 6 2 -1 7 8 9]` <br><br>
`child-two = [8 -1 7 3 4 5 -1 1 9 0]`

where -1 denotes an empty spot that needs to be filled. For finishing child-one, we go through parent-two and **the first element that we see that already doesn't exist in child-one is 4.** We notice that in the same index, we have 6 in child-one. We search through parent-two for the element 6 and we find it in the 6th index of parent-two (counting from 0 from left). We look at the corresponding position in child-one and we see that it is empty (denoted by -1). So we put 4 in that position. <br> 
Similarly, **the next element in parent-two that doesn't already exist in child-one is 5.** In the corresponding position in child-one, we have 2. We search for 2 in parent-two and we find it in the 3rd index from the left. We check the same position in child-one and we notice that it is empty, so 5 into it. As a result, child-one would be:

`child-one = [0 1 5 3 6 2 4 7 8 9]` <br>

We have the same process for filling the child-two, but this time with parent-one.

<span style="color:#55CABB">PMX is one of the most commonly used crossover operators for permutation encoded chromosomes. It is widely used in various fields, including engineering, finance, and game designing, to solve complex problems.</span>





<figure>
    <center> <img src="./pics/Crossover/pmx.png"  alt='missing' width="450"  ><center/>
<figure/>

In [7]:
import numpy as np
from copy import deepcopy 

def partially_mapped_crossover(parent_one, parent_two, pc):
    """
    This function takes two parents, and performs Partially-mapped-crossover on them. 
    parent_one: First parent
    parent_two: Second parent
    pc: The probability of crossover
    """  
    
    print("\nParents")
    print("=================================================")
    Chromosome.describe(parent_one)
    Chromosome.describe(parent_two)
    
    chrom_length = Chromosome.get_chrom_length(parent_one)
    # select two random points
    point1 = np.random.randint(1, abs(chrom_length/2))
    point2 = np.random.randint(abs(chrom_length/2), chrom_length - 1)
    # point1 = 3
    # point2 = 8

    NotSeenList1 = []
    NotSeenList2 = []

    if np.random.rand() < pc:  # if pc is greater than random number

        child_one = Chromosome(genes= np.array([-1]*chrom_length),id_=0,fitness = 125.2)   
        child_two = Chromosome(genes= np.array([-1]*chrom_length),id_=1,fitness = 125.2)

        for i in range(chrom_length):  # transfer genes
            if i >= point1 and i < point2:
                child_one.genes[i] = parent_one.genes[i]
                child_two.genes[i] = parent_two.genes[i]
        for i in range(chrom_length):
            if i < point1 or i >= point2:
                # elements that did not already transfered to child-one
                NotSeenList1.append(parent_one.genes[i])
                # elements that did not already transfered to child-two
                NotSeenList2.append(parent_two.genes[i])

        for i in range(chrom_length):  # First child
            if(child_one.genes[i] != -1 and parent_two.genes[i] in NotSeenList1):
                ans = parent_two.genes[i]
                # keep searching until you find an empty spot
                while child_one.genes[i] != -1: 
                    i = parent_two.genes.tolist().index(child_one.genes[i])
                child_one.genes[i] = ans

        for i in range(chrom_length):
            if parent_two.genes[i] in NotSeenList1:
                if(child_one.genes[i] == -1):
                    child_one.genes[i] = parent_two.genes[i]

        for i in range(chrom_length):  # Second child
            if(child_two.genes[i] != -1 and parent_one.genes[i] in NotSeenList2):
                tmp = parent_one.genes[i]
                # keep searching until you find an empty spot
                while child_two.genes[i] != -1:
                    i = parent_one.genes.tolist().index(child_two.genes[i])
                child_two.genes[i] = tmp

        for i in range(chrom_length):
            if parent_one.genes[i] in NotSeenList2:
                if(child_two.genes[i] == -1):
                    child_two.genes[i] = parent_one.genes[i]

    else:  # if pc is less than random number then don't make any change
        child_one = deepcopy(parent_one)
        child_two = deepcopy(parent_two)

    # updating fitnesses for children
    child_one.fitness = fitness_function(child_one.genes)
    child_two.fitness = fitness_function(child_two.genes)

    return child_one, child_two



chrom0 = np.array([0,1,2,3,4,5,6,7,8,9])
chrom1 = np.array([4,5,2,1,8,0,7,6,9,3])
parent_one = Chromosome(chrom0, id_= 0, fitness = fitness_function(chrom0))  
parent_two = Chromosome(chrom1, id_= 1, fitness = fitness_function(chrom1))
CROSS = partially_mapped_crossover(parent_one, parent_two, 1)

if __name__ == '__main__':
    print("\nChildren")
    print("=================================================")
    for index, _ in enumerate(CROSS):  #two children
        Chromosome.describe(CROSS[index])






Parents
ID=#0, Fitness=5, 
Genes=
[0 1 2 3 4 5 6 7 8 9]
ID=#1, Fitness=5, 
Genes=
[4 5 2 1 8 0 7 6 9 3]

Children
ID=#0, Fitness=5, 
Genes=
[8 5 2 1 4 0 7 6 9 3]
ID=#1, Fitness=5, 
Genes=
[0 1 2 3 8 5 6 7 4 9]


## <a id='cycle'>Cycle Crossover (CX)</a>

The cycle crossover operator identifies a number of so-called cycles between two parent chromosomes. Then, to form child 1, cycle one is copied from parent 1, cycle 2 from parent 2, cycle 3 from parent 1, and so on. Consider the following example. We can finish the children in 3 cycles.

<figure>
    <center> <img src="./pics/Crossover/cycle_crossover.png"  alt='missing' width="450"  ><center/>
<figure/>

**Cycle 1** <br>
`Values 8 9 0`:  We start with the first value in Parent 1 and drop down to the same position in Parent 2. 8 Goes to 0. Then, we look for 0 in Parent 1 and find it at the 10th position where we drop down to 9. Again, we look for this value in Parent 1 and find it in the 9th position and drop down to 8. Since we started with 8, we've completed our cycle.

**Cycle 2** <br>
`Values 4 1 7 2 5 6`: We start with 4 and drop down to 1. 1 is found in the 8th position in Parent 1 and we drop down to 7. 7 Drops down to 2, 2 Drops down to 5, 5 drops down to 6, and 6 drops down to 4 - Our cycle is complete.

**Cycle 3** <br>
`Value 3`: The only possible cycle left is of length 1 and contains the value 3.

Filling in the offspring: <br>
`Copy Cycle 1:` Cycle 1 values from Parent 1 are copied to Child 1, and values from Parent 2 will be copied to Child 2. Cycle 2 will by different. <br>
`Copy Cycle 2:` Cycle 2 values from Parent 1 will be copied to Child 2, and values from Parent 1 will be copied to Child 1. <br>
`Copy Cycle 3:` Cycle 3 is like Cycle 1, Parent 1 goes to Child 1, Parent 2 goes to Child 2.

This alternating process goes on until the children are finished. <span style="color:#55CABB">This crossover is used for problems such as the traveling salesman, to find the shortest possible route, over generations.</span>


In [8]:
import numpy as np
from copy import deepcopy 

def cycle_crossover(parent_one, parent_two, pc):
    """
    This function takes two parents, and performs Cycle crossover on them. 
    parent_one: First parent
    parent_two: Second parent
    pc: The probability of crossover
    """

    chrom_length = Chromosome.get_chrom_length(parent_one)

    child_one = Chromosome(genes= np.array([-1]*chrom_length),id_=0,fitness = 125.2)   
    child_two = Chromosome(genes= np.array([-1]*chrom_length),id_=1,fitness = 125.2)

    print("\nParents")
    print("=================================================")
    Chromosome.describe(parent_one)
    Chromosome.describe(parent_two)
    
    child_one = Chromosome(genes=np.array([-1] * chrom_length), id_=0, fitness=125.2)
    child_two = Chromosome(genes=np.array([-1] * chrom_length), id_=1, fitness=125.2)

    if np.random.rand() < pc:  # if pc is greater than random number
        p1_copy = parent_one.genes.tolist()
        p2_copy = parent_two.genes.tolist()
        swap = True
        count = 0
        pos = 0

        while True:
            if count > chrom_length:
                break
            for i in range(chrom_length):
                if child_one.genes[i] == -1:
                    pos = i
                    break

            if swap:
                while True:
                    child_one.genes[pos] = parent_one.genes[pos]
                    count += 1
                    pos = parent_two.genes.tolist().index(parent_one.genes[pos])
                    if p1_copy[pos] == -1:
                        swap = False
                        break
                    p1_copy[pos] = -1
            else:
                while True:
                    child_one.genes[pos] = parent_two.genes[pos]
                    count += 1
                    pos = parent_one.genes.tolist().index(parent_two.genes[pos])
                    if p2_copy[pos] == -1:
                        swap = True
                        break
                    p2_copy[pos] = -1

        for i in range(chrom_length): #for the second child
            if child_one.genes[i] == parent_one.genes[i]:
                child_two.genes[i] = parent_two.genes[i]
            else:
                child_two.genes[i] = parent_one.genes[i]

        for i in range(chrom_length): #Special mode
            if child_one.genes[i] == -1:
                if p1_copy[i] == -1: #it means that the ith gene from p1 has been already transfered
                    child_one.genes[i] = parent_two.genes[i]
                else:
                    child_one.genes[i] = parent_one.genes[i]

    else:  # if pc is less than random number then don't make any change
        child_one = deepcopy(parent_one)
        child_two = deepcopy(parent_two)

    # updating fitnesses for children
    child_one.fitness = fitness_function(child_one.genes)
    child_two.fitness = fitness_function(child_two.genes)

    return child_one, child_two



chrom0 = np.array([0,1,2,3,4,5,6,7,8,9])
chrom1 = np.array([4,5,2,1,8,0,7,6,9,3])
parent_one = Chromosome(chrom0, id_= 0, fitness = fitness_function(chrom0))  
parent_two = Chromosome(chrom1, id_= 1, fitness = fitness_function(chrom1))
CROSS = cycle_crossover(parent_one, parent_two, 1)

if __name__ == '__main__':
    print("\nChildren")
    print("=================================================")
    for index, _ in enumerate(CROSS):  #two children
        Chromosome.describe(CROSS[index])







Parents
ID=#0, Fitness=5, 
Genes=
[0 1 2 3 4 5 6 7 8 9]
ID=#1, Fitness=5, 
Genes=
[4 5 2 1 8 0 7 6 9 3]

Children
ID=#0, Fitness=5, 
Genes=
[0 1 2 3 4 5 6 7 8 9]
ID=#1, Fitness=5, 
Genes=
[4 5 2 1 8 0 7 6 9 3]


## <a id='intermediate'>Heuristic Crossover (HC) or Intermediate Crossover (IC)</a>

In this crossover two parents are selected from the population and one offspring is produced. We first create a random number between 0 and 1. Then the genes of the child chromosome will be calculated as follows: <br>

`child_one[i] = parent_one[i] + a*(parent_two[i] - parent_one[i])`

where variable `a` is a random number between 0 and 1.

<figure>
    <center> <img src="./pics/Crossover/heuristic_crossover.png"  alt='missing' width="450"  ><center/>
<figure/>

In [9]:
import numpy as np
from copy import deepcopy

def heuristic_crossover(parent_one, parent_two, pc):
    """
    This function takes two parents, and performs Heuristic crossover on them. 
    parent_one: First parent
    parent_two: Second parent
    pc: The probability of crossover
    """  

    print("\nParents")
    print("=================================================")
    Chromosome.describe(parent_one)
    Chromosome.describe(parent_two)
    
    chrom_length = Chromosome.get_chrom_length(parent_one)
    child_one = Chromosome(genes= np.array([0.0]*chrom_length),id_=0,fitness = 125.2)   
    # child_two = chromosome(genes= np.array([-1]*chrom_length),id=1,fitness = 125.2)
    a = np.random.uniform(low=0, high=1)
    print(a)
    if np.random.rand() < pc:  # if pc is greater than random number
        for i in range(chrom_length):
            child_one.genes[i] = round(parent_one.genes[i] + a*np.abs(parent_two.genes[i] - parent_one.genes[i]), 3)

    else:  # if pc is less than random number then don't make any change
        child_one = deepcopy(parent_one)
        
    # updating fitnesses for children
    child_one.fitness = fitness_function(child_one.genes)

    return child_one



chrom0 = np.array([0.121, 0.152, 0.231, 0.143, 0.732, 0.315, 0.434, 0.633])
chrom1 = np.array([0.765, 0.168, 0.914, 0.435, 0.893, 0.332, 0.981, 0.194])
parent_one = Chromosome(chrom0, id_= 0, fitness = fitness_function(chrom0))  
parent_two = Chromosome(chrom1, id_= 1, fitness = fitness_function(chrom1))
CROSS = heuristic_crossover(parent_one, parent_two, 1)

if __name__ == '__main__':
    print("\nChildren")
    print("=================================================")
    Chromosome.describe(CROSS)



Parents
ID=#0, Fitness=6, 
Genes=
[0.121 0.152 0.231 0.143 0.732 0.315 0.434 0.633]
ID=#1, Fitness=4, 
Genes=
[0.765 0.168 0.914 0.435 0.893 0.332 0.981 0.194]
0.5924962257560331

Children
ID=#0, Fitness=3, 
Genes=
[0.503 0.161 0.636 0.316 0.827 0.325 0.758 0.893]


## <a id='shuffle'>Shuffle Crossover (SX)</a>

In this crossover, we select two parents from the population and two offspring are produced. We choose several random positions which are the same in both parents. Then we shuffle the genes in these positions in the same way in both parents. After that, we do one-point crossover on the parents and finally unshuffle the genes in the offspring that we shuffled earlier.

<figure>
    <center> <img src="./pics/Crossover/shuffle_crossover.png"  alt='missing' width="700"  ><center/>
<figure/>

In [10]:
import numpy as np
from copy import deepcopy 

def shuffle_crossover(parent_one, parent_two, pc):
    """
    This function takes two parents, and performs Shuffle crossover on them. 
    parent_one: First parent
    parent_two: Second parent
    pc: The probability of crossover
    """  
    
    print("\nParents")
    print("=================================================")    
    Chromosome.describe(parent_one)
    Chromosome.describe(parent_two)
    
    chrom_length = Chromosome.get_chrom_length(parent_one)
    child_one = Chromosome(genes= np.array([-1]*chrom_length),id_=0,fitness = 125.2)   
    child_two = Chromosome(genes= np.array([-1]*chrom_length),id_=1,fitness = 125.2)
    points = sorted_random_numbers(4,chrom_length)
    # print(points)
  
    if np.random.rand() < pc:  # if pc is greater than random number
        # shuffling
        parent_one.genes[points[0]], parent_one.genes[points[2]] = parent_one.genes[points[2]], parent_one.genes[points[0]] #shuffle the elements of p1
        parent_one.genes[points[1]], parent_one.genes[points[3]] = parent_one.genes[points[3]], parent_one.genes[points[1]]

        parent_two.genes[points[0]], parent_two.genes[points[2]] = parent_two.genes[points[2]], parent_two.genes[points[0]] #shuffle the elements of p2 in the same way
        parent_two.genes[points[1]], parent_two.genes[points[3]] = parent_two.genes[points[3]], parent_two.genes[points[1]]

        # performing one point crossover
        print("\nParents after performing shuffles:")
        child_one, child_two = one_point_crossover(parent_one,parent_two, 0,1) 

        # unshuffling
        child_one.genes[points[0]], child_one.genes[points[2]] = child_one.genes[points[2]], child_one.genes[points[0]] #reshuffle the elements of c1
        child_one.genes[points[1]], child_one.genes[points[3]] = child_one.genes[points[3]], child_one.genes[points[1]]

        child_two.genes[points[0]], child_two.genes[points[2]] = child_two.genes[points[2]], child_two.genes[points[0]] #reshuffle the elements of c2 in the same way
        child_two.genes[points[1]], child_two.genes[points[3]] = child_two.genes[points[3]], child_two.genes[points[1]]
        
    else:  # if pc is less than random number then don't make any change
        child_one = deepcopy(parent_one)
        child_two = deepcopy(parent_two)
    return child_one, child_two



chrom0 = np.array([1, 1, 1, 0, 1, 0, 0, 1, 0])
chrom1 = np.array([1, 0, 0, 0, 1, 0, 1, 1, 0])
parent_one = Chromosome(chrom0, id_= 0, fitness = fitness_function(chrom0))  
parent_two = Chromosome(chrom1, id_= 1, fitness = fitness_function(chrom1))
CROSS = shuffle_crossover(parent_one, parent_two, 1)

if __name__ == '__main__':
    print("\nChildren")
    print("=================================================")
    for index, _ in enumerate(CROSS):  #two children
        Chromosome.describe(CROSS[index])




Parents
ID=#0, Fitness=5, 
Genes=
[1 1 1 0 1 0 0 1 0]
ID=#1, Fitness=4, 
Genes=
[1 0 0 0 1 0 1 1 0]

Parents after performing shuffles:

Parents
ID=#0, Fitness=5, 
Genes=
[1 0 1 0 1 0 1 1 0]
ID=#1, Fitness=4, 
Genes=
[1 1 0 0 1 0 0 1 0]

Children
ID=#0, Fitness=4, 
Genes=
[1 0 0 0 1 0 1 1 0]
ID=#1, Fitness=5, 
Genes=
[1 1 1 0 1 0 0 1 0]


## <a id='reduce-surrogate'>Reduce Surrogate Crossover (RSX)</a>

This crossover happens only if the selected parents are different in more than one gene. This means that if the parents are similar or are different in only one gene, this crossover won't be operated on the parents. If the value of more than one gene is different, the positions where the genes of the parents have different values are selected and one of these positions is chosen randomly and one-point crossover is operated on parents in the chosen position.

<figure>
    <center> <img src="./pics/Crossover/reduced_surrogate_crossover.png"  alt='missing' width="450"  ><center/>
<figure/>

In [11]:
import numpy as np
from copy import deepcopy

def reduced_surragate_crossover(parent_one, parent_two, pc):
    """
    This function takes two parents, and performs Reduced surragate crossover on them. 
    parent_one: First parent
    parent_two: Second parent
    pc: The probability of crossover
    """  
   
    chrom_length = Chromosome.get_chrom_length(parent_one)
    child_one = Chromosome(genes= np.array([-1]*chrom_length),id_=0,fitness = 125.2)   
    child_two = Chromosome(genes= np.array([-1]*chrom_length),id_=1,fitness = 125.2)
    dif = 0

    for i in range(chrom_length):
        if parent_one.genes[i] != parent_two.genes[i]:
            dif+=1
  
    # if pc is greater than random number and more that one gene is different
    if np.random.rand() < pc and dif>1:  
        # choose number of points to be nominated for crossover points
        point_nums = np.random.randint(2, abs(chrom_length)) 
        # choose several positions for crossover
        points = sorted_random_numbers(point_nums, chrom_length)
        # choose the final point for 1PX 
        pos = np.random.randint(0, point_nums) 
        
        print("The chosen point for one_point_crossover:" , points[pos])
        print("\nParents in one_point_crossover:")
        child_one, child_two = one_point_crossover(parent_one, parent_two, points[pos], 1)        

    else:  # if pc is less than random number then don't make any change
        child_one = deepcopy(parent_one)
        child_two = deepcopy(parent_two)

    # updating fitnesses for children
    child_one.fitness = fitness_function(child_one.genes)
    child_two.fitness = fitness_function(child_two.genes)

    return child_one, child_two



chrom0 = np.array([1, 1, 1, 0, 1, 0, 0, 1, 0])
chrom1 = np.array([1, 0, 0, 0, 1, 0, 1, 1, 0])
parent_one = Chromosome(chrom0, id_= 0, fitness = fitness_function(chrom0))  
parent_two = Chromosome(chrom1, id_= 1, fitness = fitness_function(chrom1))
CROSS = reduced_surragate_crossover(parent_one, parent_two, 1)

if __name__ == '__main__':
    print("\nChildren")
    print("=================================================")
    for index, _ in enumerate(CROSS):  #two children
        Chromosome.describe(CROSS[index])


The chosen point for one_point_crossover: 1

Parents in one_point_crossover:

Parents
ID=#0, Fitness=5, 
Genes=
[1 1 1 0 1 0 0 1 0]
ID=#1, Fitness=4, 
Genes=
[1 0 0 0 1 0 1 1 0]

Children
ID=#0, Fitness=4, 
Genes=
[1 0 0 0 1 0 1 1 0]
ID=#1, Fitness=5, 
Genes=
[1 1 1 0 1 0 0 1 0]


## <a id='average'>Average Crossover (AX)</a>

In average crossover, the genes of the offspring in each position are calculated by taking the arithmetic mean of both of its parents in that position. For integer representation, the floor of the arithmetic mean is considered for the value of the gene in the child.

`child_one[i] = floor((parent_one[i] + parent_two[i]) / 2)`

<figure>
    <center> <img src="./pics/Crossover/average_crossover.png"  alt='missing' width="450"  ><center/>
<figure/>

In [12]:
import numpy as np
from copy import deepcopy

def average_crossover(parent_one, parent_two, pc):
    """
    This function takes two parents, and performs Average crossover on them. 
    parent_one: First parent
    parent_two: Second parent
    pc: The probability of crossover
    """  

    print("\nParents")
    print("=================================================")    
    Chromosome.describe(parent_one)
    Chromosome.describe(parent_two)
    
    chrom_length = Chromosome.get_chrom_length(parent_one)
    child_one = Chromosome(genes= np.array([-1]*chrom_length),id_=0,fitness = 125.2)   
    # child_two = chromosome(genes= np.array([-1]*chrom_length),id=1,fitness = 125.2)
  
    if np.random.rand() < pc:  # if pc is greater than random number
        for i in range(chrom_length):
            child_one.genes[i] = int((parent_one.genes[i] + parent_two.genes[i])/2)

    else:  # if pc is less than random number then don't make any change
        child_one = deepcopy(parent_one)

    # updating fitnesses for children
    child_one.fitness = fitness_function(child_one.genes)

    return child_one



chrom0 = np.array([5, 3, 3, 2, 3, 9, 7, 6, 5, 4])
chrom1 = np.array([5, 4, 7, 6, 5, 2, 6, 1, 3, 0])
parent_one = Chromosome(chrom0, id_= 0, fitness = fitness_function(chrom0))  
parent_two = Chromosome(chrom1, id_= 1, fitness = fitness_function(chrom1))
CROSS = average_crossover(parent_one, parent_two, 1)

if __name__ == '__main__':
    print("\nChildren")
    print("=================================================")
    Chromosome.describe(CROSS)



Parents
ID=#0, Fitness=5, 
Genes=
[5 3 3 2 3 9 7 6 5 4]
ID=#1, Fitness=5, 
Genes=
[5 4 7 6 5 2 6 1 3 0]

Children
ID=#0, Fitness=6, 
Genes=
[5 3 5 4 4 5 6 3 4 2]


## <a id='discrete'>Discrete Crossover (DC)</a>

In discrete crossover, a uniform random real number is produced for each gene of the parent's chromosome. If this number is less than 0.5, then the gene of the offspring in that position will be filled by parent one. Otherwise, the gene of the offspring in that position is filled by parent two. The main difference between discrete crossover and uniform crossover is that in discrete crossover we don't calculate a mask vector for the crossover to take place and also one offspring is produced instead of two.

<figure>
    <center> <img src="./pics/Crossover/discrete_crossover.png"  alt='missing' width="450"  ><center/>
<figure/>

In [13]:
import numpy as np
from copy import deepcopy 

def discrete_crossover(parent_one, parent_two, pc):
    """
    This function takes two parents, and performs Discrete crossover on them. 
    parent_one: First parent
    parent_two: Second parent
    pc: The probability of crossover
    """  

    print("\nParents")
    print("=================================================")    
    Chromosome.describe(parent_one)
    Chromosome.describe(parent_two)
    
    chrom_length = Chromosome.get_chrom_length(parent_one)
    child_one = Chromosome(genes= np.array([-1]*chrom_length),id_=0,fitness = 125.2)  # child
  
    if np.random.rand() < pc:  # if pc is greater than random number
        for i in range(chrom_length):
            w = np.random.uniform(low=0, high=1)
            if w<0.5:
                child_one.genes[i] = parent_one.genes[i]
            else:
                child_one.genes[i] = parent_two.genes[i]

    else:  # if pc is less than random number then don't make any change
        child_one = deepcopy(parent_one)
    
    # updating fitnesses for children
    child_one.fitness = fitness_function(child_one.genes)

    return child_one



chrom0 = np.array([1, 1, 1, 0, 1, 0, 0, 1, 0, 1])
chrom1 = np.array([1, 0, 0, 0, 1, 0, 1, 1, 0, 0])
parent_one = Chromosome(chrom0, id_= 0, fitness = fitness_function(chrom0))  
parent_two = Chromosome(chrom1, id_= 1, fitness = fitness_function(chrom1))
CROSS = discrete_crossover(parent_one, parent_two, 1)

if __name__ == '__main__':
    print("\nChildren")
    print("=================================================")
    Chromosome.describe(CROSS)



Parents
ID=#0, Fitness=6, 
Genes=
[1 1 1 0 1 0 0 1 0 1]
ID=#1, Fitness=4, 
Genes=
[1 0 0 0 1 0 1 1 0 0]

Children
ID=#0, Fitness=5, 
Genes=
[1 0 1 0 1 0 1 1 0 0]


## <a id='flat'>Flat Crossover (FC)</a>
In flat crossover, for each gene of the child, we produce a uniform random real number w between the values of the parents and we transfer this number to the child. In other words, if the parent-one is the first parent and the parent-two is the second parent, then the value for the gene `i` of the child-one is calculated as follows:

`child_one[i] ~ (minimum(parent_one[i], parent_two[i]), maximum(parent_one[i], parent_two[i]))`

<figure>
    <center> <img src="./pics/Crossover/flat_crossover.png"  alt='missing' width="450"  ><center/>
<figure/>

In [14]:
import numpy as np
from copy import deepcopy

def flat_crossover(parent_one, parent_two, pc):
    """
    This function takes two parents, and performs Flat crossover on them. 
    parent_one: First parent
    parent_two: Second parent
    pc: The probability of crossover
    """  

    print("\nParents")
    print("=================================================")    
    Chromosome.describe(parent_one)
    Chromosome.describe(parent_two)
    
    chrom_length = Chromosome.get_chrom_length(parent_one)
    child_one = Chromosome(genes= np.array([0.0]*chrom_length),id_=0,fitness = 125.2)   
  
    if np.random.rand() < pc:  # if pc is greater than random number
        for i in range(chrom_length):
            w = np.random.uniform(low= np.minimum(parent_one.genes[i], parent_two.genes[i]), high= np.maximum(parent_one.genes[i], parent_two.genes[i]))
            child_one.genes[i] = round(w, 3)

    else:  # if pc is less than random number then don't make any change
        child_one = deepcopy(parent_one)

    # updating fitnesses for children
    child_one.fitness = fitness_function(child_one.genes)

    return child_one



chrom0 = np.array([0.121, 0.152, 0.231, 0.143, 0.732, 0.315, 0.434, 0.633])
chrom1 = np.array([0.765, 0.168, 0.914, 0.435, 0.893, 0.332, 0.981, 0.194])
parent_one = Chromosome(chrom0, id_= 0, fitness = fitness_function(chrom0))  
parent_two = Chromosome(chrom1, id_= 1, fitness = fitness_function(chrom1))
CROSS = flat_crossover(parent_one, parent_two, 1)

if __name__ == '__main__':
    print("\nChildren")
    print("=================================================")
    Chromosome.describe(CROSS)



Parents
ID=#0, Fitness=6, 
Genes=
[0.121 0.152 0.231 0.143 0.732 0.315 0.434 0.633]
ID=#1, Fitness=4, 
Genes=
[0.765 0.168 0.914 0.435 0.893 0.332 0.981 0.194]

Children
ID=#0, Fitness=4, 
Genes=
[0.167 0.164 0.604 0.241 0.755 0.332 0.677 0.612]


## <a id='multivariate'>Multivariate Crossover</a>

In multivariate crossover, we divide the parents into several substrings. For each substring, we produce a uniform random real number and we check whether that number is less than PC (crossover probability). If the random number produced is less than PC, then we operate one-point crossover for that substring and we transfer the resulted genes to the children. Otherwise, the genes from the parents in that substring are directly transferred from the parents to the offspring.

<figure>
    <center> <img src="./pics/Crossover/multivariate_crossover.png"  alt='missing' width="500"  ><center/>
<figure/>

In [15]:
import numpy as np
from copy import deepcopy 

def multivariate_crossover(parent_one, parent_two, pc):
    """
    This function takes two parents, and performs Multivariate crossover on them. 
    parent_one: First parent
    parent_two: Second parent
    pc: The probability of crossover
    """  

    print("\nParents")
    print("=================================================")    
    Chromosome.describe(parent_one)
    Chromosome.describe(parent_two)
    
    chrom_length = Chromosome.get_chrom_length(parent_one)
    child_one = Chromosome(genes= np.array([-1]*chrom_length),id_=0,fitness = 125.2)   
    child_two = Chromosome(genes= np.array([-1]*chrom_length),id_=1,fitness = 125.2)
    random_nums = []

    if np.random.rand() < pc:  # if pc is greater than random number
        points = sorted_random_numbers(2,chrom_length) #selecting two points
        for i in range(3): # selecting three random numbers for each substring
            random_nums.append(np.random.rand())

    #************************************************************************ part1
        if random_nums[0]<0.5:
            for i in range(points[0]):
                child_one.genes[i] = parent_one.genes[i]
                child_two.genes[i] = parent_two.genes[i]
        else:
            parent1_sub = Chromosome(genes= [-1]*points[0],id_=0,fitness = 125.2)
            parent2_sub = Chromosome(genes= [-1]*points[0],id_=1,fitness = 125.2)

            for i in range(points[0]):
                parent1_sub.genes[i] = parent_one.genes[i]
                parent2_sub.genes[i] = parent_two.genes[i]


            parent1_sub, parent2_sub = one_point_crossover(parent1_sub,parent2_sub,0,1) #perform 1PX

            for i in range(points[0]): #transfer the genes to children
                child_one.genes[i] = parent1_sub.genes[i]
                child_two.genes[i] = parent2_sub.genes[i]

    #************************************************************************part2
        if random_nums[1]<0.5:
            for i in range(points[0],points[1]):
                child_one.genes[i] = parent_one.genes[i]
                child_two.genes[i] = parent_two.genes[i]
        else:
            parent1_sub = Chromosome(genes= [-1]*(points[1]-points[0]),id_=0,fitness = 125.2)
            parent2_sub = Chromosome(genes= [-1]*(points[1]-points[0]),id_=1,fitness = 125.2)
            k = 0
            for i in range(points[0],points[1]):
                parent1_sub.genes[k] = parent_one.genes[i]
                parent2_sub.genes[k] = parent_two.genes[i]
                k+=1

            parent1_sub, parent2_sub = one_point_crossover(parent1_sub,parent2_sub,0,1) #perform 1PX
            k = 0
            for i in range(points[0],points[1]): #transfer the genes to children
                child_one.genes[i] = parent1_sub.genes[k]
                child_two.genes[i] = parent2_sub.genes[k]
                k+=1

    #************************************************************************part3
        if random_nums[2]<0.5:
            for i in range(points[1],chrom_length):
                child_one.genes[i] = parent_one.genes[i]
                child_two.genes[i] = parent_two.genes[i]
        else:
            parent1_sub = Chromosome(genes= [-1]*(chrom_length-points[1]),id_=0,fitness = 125.2)
            parent2_sub = Chromosome(genes= [-1]*(chrom_length-points[1]),id_=1,fitness = 125.2)
            k = 0
            for i in range(points[1],chrom_length):
                parent1_sub.genes[k] = parent_one.genes[i]
                parent2_sub.genes[k] = parent_two.genes[i]
                k+=1

            parent1_sub, parent2_sub = one_point_crossover(parent1_sub,parent2_sub,0,1) #perform 1PX
            k = 0
            for i in range(points[1],chrom_length): #transfer the genes to children
                child_one.genes[i] = parent1_sub.genes[k]
                child_two.genes[i] = parent2_sub.genes[k]
                k+=1     
      
    else:  # if pc is less than random number then don't make any change
        child_one = deepcopy(parent_one)
        child_two = deepcopy(parent_two)

    # updating fitnesses for children
    child_one.fitness = fitness_function(child_one.genes)
    child_two.fitness = fitness_function(child_two.genes)

    return child_one, child_two



chrom0 = np.array([1, 1, 1, 0, 1, 0, 0, 1, 0])
chrom1 = np.array([1, 0, 0, 0, 1, 0, 1, 1, 0])
parent_one = Chromosome(chrom0, id_= 0, fitness = fitness_function(chrom0))  
parent_two = Chromosome(chrom1, id_= 1, fitness = fitness_function(chrom1))
CROSS = multivariate_crossover(parent_one, parent_two, 1)

if __name__ == '__main__':
    print("\nChildren")
    print("=================================================")
    for index, _ in enumerate(CROSS):  #two children
        Chromosome.describe(CROSS[index])




Parents
ID=#0, Fitness=5, 
Genes=
[1 1 1 0 1 0 0 1 0]
ID=#1, Fitness=4, 
Genes=
[1 0 0 0 1 0 1 1 0]

Parents
ID=#0, Fitness=125.2, 
Genes=
[0, 1, 0]
ID=#1, Fitness=125.2, 
Genes=
[1, 1, 0]

Children
ID=#0, Fitness=5, 
Genes=
[1 1 1 0 1 0 0 1 0]
ID=#1, Fitness=4, 
Genes=
[1 0 0 0 1 0 1 1 0]


## <a id='count-preserving'>Count Preserving Crossover (CPC)</a>

**Count-preserving crossover (CPC)** is a genetic algorithm operator that aims to preserve the number of chromosomes equal to 1 in each chromosome in the initial population. <span style="color:#55CABB">This operator is used to create offspring for the new population. It is used for problems such as the **traveling salesman problem**, to find the shortest possible route over generations. CPC is used in single-objective numerical optimization problems.</span>

The CPC operator selects two parents from the parent pool and creates two lists of differences between them. The operator then tries to preserve the same number of genes equal to 1 in each chromosome in the initial population. Here's an example:

```
Parent 1: 1 0 1 0 1 1 1 0
Parent 2: 1 1 1 0 1 0 0 1
up: 5, 6
down: 1, 7 
```

where up is the positions where the gene of parent-one has a value of 1 and the gene of parent-two has a value of 0; down is the positions where parent-two is 1 and parent-one is 0. The operator then creates two offspring by swapping the values in the up and down lists between the two parents. The resulting offspring are:

```
Offspring 1: 1 1 1 0 1 0 0 1
Offspring 2: 1 0 1 0 1 1 1 0
```

As you can see, the offspring 1 is the same as parent 2, and offspring 2 is the same as parent 1; so we shuffle these values a little bit to avoid getting the same chromosomes over and over. So, the final offspring could be:

```
Offspring 1: 1 0 1 0 1 1 0 1
Offspring 2: 1 0 1 0 1 0 1 1
```

<figure>
    <center> <img src="./pics/Crossover/cpx.png"  alt='missing' width="500"  ><center/>
<figure/>

In [16]:
import numpy as np
from copy import deepcopy 

def count_preserving_crossover(parent_one, parent_two, pc):
    """
    This function takes two parents, and performs Count-preserving crossover on them.
    In this crossover, the number of ones must be perserved in offsprings. We assume that 
    the number of ones are equal in both parents. 
    parent_one: First parent
    parent_two: Second parent
    pc: The probability of crossover
    """  

    print("\nParents")
    print("=================================================")    
    Chromosome.describe(parent_one)
    Chromosome.describe(parent_two)
    
    chrom_length = Chromosome.get_chrom_length(parent_one)
    child_one = Chromosome(genes= np.array([-1]*chrom_length),id_=0,fitness = 125.2)   
    child_two = Chromosome(genes= np.array([-1]*chrom_length),id_=1,fitness = 125.2)
    up = [] #contains the positions of bits in the list which parent_one is one and parent_two is zero
    down = [] #contains the positions of bits in the list which parent_one is zero and parent_two is one

    if np.random.rand() < pc:  # if pc is greater than random number

        # copying parents to children
        for i in range(chrom_length): 
            child_one.genes[i] = parent_one.genes[i]
            child_two.genes[i] = parent_two.genes[i]

        for i in range(chrom_length):
            if parent_one.genes[i] != parent_two.genes[i]:
                if parent_one.genes[i] == 1 and parent_two.genes[i] == 0:
                    up.append(i)
                else:
                    down.append(i)
                    
        
        positions = up + down
        positions.sort()

        print(up)
        print(down)

        for i in range(len(up)): #changing the elements in the positions
            child_one.genes[up[i]], child_one.genes[down[i]] = child_one.genes[down[i]], child_one.genes[up[i]] 
            child_two.genes[up[i]], child_two.genes[down[i]] = child_two.genes[down[i]], child_two.genes[up[i]] 

        #==================================================
        # shuffling the lists of genes in children 
        #==================================================
        child_1 = child_one.genes
        child_2 = child_two.genes
        # Create a new list with the values at the specified positions
        values1 = [child_1[i] for i in positions]
        values2 = [child_2[i] for i in positions]

        # shuffle the lists to avoid producing similar children 
        np.random.shuffle(values1)
        np.random.shuffle(values2)

        # Replace the values in the original list with the shuffled values
        for i, j in enumerate(positions):
            child_1[j] = values1[i]
            child_2[j] = values2[i]

        child_one.genes = child_1
        child_two.genes = child_2
      
    else:  # if pc is less than random number then don't make any change
        child_one = deepcopy(parent_one)
        child_two = deepcopy(parent_two)

    # updating fitnesses for children
    child_one.fitness = fitness_function(child_one.genes)
    child_two.fitness = fitness_function(child_two.genes)

    return child_one, child_two


chrom0 = np.array([0, 1, 1, 1, 0, 1, 0, 1, 1, 0]) # same number of 1s
chrom1 = np.array([1, 0, 0, 0, 1, 1, 1, 1, 1, 0])
parent_one = Chromosome(chrom0, id_= 0, fitness = fitness_function(chrom0))  
parent_two = Chromosome(chrom1, id_= 1, fitness = fitness_function(chrom1))
CROSS = count_preserving_crossover(parent_one, parent_two, 1)

if __name__ == '__main__':
    print("\nChildren")
    print("=================================================")
    for index, _ in enumerate(CROSS):  #two children
        Chromosome.describe(CROSS[index])





Parents
ID=#0, Fitness=6, 
Genes=
[0 1 1 1 0 1 0 1 1 0]
ID=#1, Fitness=6, 
Genes=
[1 0 0 0 1 1 1 1 1 0]
[1, 2, 3]
[0, 4, 6]

Children
ID=#0, Fitness=6, 
Genes=
[1 0 0 0 1 1 1 1 1 0]
ID=#1, Fitness=6, 
Genes=
[1 0 1 0 1 1 0 1 1 0]


## <a id='modified-order'>Modified Order Crossover (MOC)</a>

In modified order crossover, we choose a random position and then we find the left side of parent-two in parent-one and transfer these genes from parent-one to child-one. Then we loop over the right side of the parent-two and fill the empty positions of child-one from left to right in the same order. We do the same thing for child-two by finding the elements of the left side of parent-one in parent-two and transferring them; and then filling the empty spots with the elements on the right side of parent-one.

<figure>
    <center> <img src="./pics/Crossover/moc.png"  alt='missing' width="450"  ><center/>
<figure/>

In [42]:
import numpy as np
from copy import deepcopy 

def modified_order_crossover(parent_one, parent_two, pc):
    """
    This function takes two parents, and performs Modified order crossover on them. 
    parent_one: First parent
    parent_two: Second parent
    pc: The probability of crossover
    """  
    
    print("\nParents")
    print("=================================================")    
    Chromosome.describe(parent_one)
    Chromosome.describe(parent_two)
    
    chrom_length = Chromosome.get_chrom_length(parent_one)
    child_one = Chromosome(genes= np.array([-1]*chrom_length),id_=0,fitness = 125.2)   
    child_two = Chromosome(genes= np.array([-1]*chrom_length),id_=1,fitness = 125.2)

    if np.random.rand() < pc:  # if pc is greater than random number
        hold1 = [] #for holding the elements in the one half of the parent_two
        hold2 = [] #for holding the elements in the one half of the parent_one

        pos = np.random.randint(2, chrom_length)
        # pos = 4
        for i in range(pos):
            hold1.append(parent_two.genes[i]) #first half of the parent_two
            hold2.append(parent_one.genes[i]) #first half of the parent_one
        for i in range(chrom_length):
            if parent_one.genes[i] in hold1:
                child_one.genes[i] = parent_one.genes[i]
            if parent_two.genes[i] in hold2:
                child_two.genes[i] = parent_two.genes[i]

        hold1 = []
        hold2 = []
        for i in range(pos,chrom_length):
            hold1.append(parent_two.genes[i]) #second half of the parent_two
            hold2.append(parent_one.genes[i]) #second half of the parent_one

        k = 0
        m = 0
        for i in range(chrom_length):
            if child_one.genes[i] == -1:
                child_one.genes[i] = hold1[k]
                k+=1
            if child_two.genes[i] == -1:
                child_two.genes[i] = hold2[m]
                m+=1

    else:  # if pc is less than random number then don't make any change
        child_one = deepcopy(parent_one)
        child_two = deepcopy(parent_two)

    # updating fitnesses for children
    child_one.fitness = fitness_function(child_one.genes)
    child_two.fitness = fitness_function(child_two.genes)

    return child_one, child_two



chrom0 = np.array([0, 1, 2, 3, 4, 6, 9, 8, 5, 7])
chrom1 = np.array([2, 1, 9, 8, 0, 5, 6, 3, 7, 4])
parent_one = Chromosome(chrom0, id_= 0, fitness = fitness_function(chrom0))  
parent_two = Chromosome(chrom1, id_= 1, fitness = fitness_function(chrom1))
CROSS = modified_order_crossover(parent_one, parent_two, 1)

if __name__ == '__main__':
    print("\nChildren")
    print("=================================================")
    for index, _ in enumerate(CROSS):  #two children
        Chromosome.describe(CROSS[index])




Parents
ID=#0, Fitness=5, 
Genes=
[0 1 2 3 4 6 9 8 5 7]
ID=#1, Fitness=5, 
Genes=
[2 1 9 8 0 5 6 3 7 4]

Children
ID=#0, Fitness=5, 
Genes=
[9 1 2 8 0 5 6 3 7 4]
ID=#1, Fitness=5, 
Genes=
[2 1 3 4 0 6 9 8 5 7]


## <a id='random-respectful'>Random Respectful Crossover (RRC)</a>

In random respectful crossover, we use a list named `similarity vector` to produce the offspring. To calculate `similarity vector`, we compare the genes of the two parents in each position. If the value of the genes in that position is the same, the value of the similarity vector for that position becomes the value of the genes in that position. Otherwise, the value of the similarity vector for that position becomes -1. To calculate the values of the genes for the offspring we do the following.

<ul>
<li>If the value of the similarity vector in position i is 0, the value of the genes in that position for both offspring becomes 0.</li>
<li>If the value of the similarity vector in position i is 1, the value of the genes in that position for both offspring becomes 1.</li>
<li>If the value of the similarity vector in position i is -1, we produce a uniform random real number `w` between 0 and 1 to determine what the value of the offspring in that position would be.</li>
    <ul type="circle">
        <li>If w < 0.5, the value of the gene in that position becomes 1;</li>
        <li>otherwise, the value of the gene in that position becomes 0. </li>
    </ul>
</ul>

<figure>
    <center> <img src="./pics/Crossover/random_respectful_crossover.png"  alt='missing' width="450"  ><center/>
<figure/>

In [18]:
import numpy as np
from copy import deepcopy

def calculate_similarity_vector(chrom1, chrom2):
    """
    To calculate the similarity-vector for two individuals.
    chrom1: First individual
    chrom2: Second individual
    """  
    chrom_length = Chromosome.get_chrom_length(chrom1)
    similarity_vector = [-1]*chrom_length
    for i in range(chrom_length):
        if chrom1.genes[i] == chrom2.genes[i]:
            if parent_one.genes[i] == 0:
                similarity_vector[i] = 0
            else:
                similarity_vector[i] = 1
    return similarity_vector


def random_respectful_crossover(parent_one, parent_two, pc):
    """
    This function takes two parents, and performs Random respectful crossover on them. 
    parent_one: First parent
    parent_two: Second parent
    pc: The probability of crossover
    """  

    print("\nParents")
    print("=================================================")
    Chromosome.describe(parent_one)
    Chromosome.describe(parent_two)
    
    chrom_length = Chromosome.get_chrom_length(parent_one)
    child_one = Chromosome(genes= np.array([-1]*chrom_length),id_=0,fitness = 125.2)   
    child_two = Chromosome(genes= np.array([-1]*chrom_length),id_=1,fitness = 125.2)

    if np.random.rand() < pc:  # if pc is greater than random number
        
        similarity_vector = calculate_similarity_vector(parent_one, parent_two)
        print("\nSimilarity-vector:", similarity_vector)

        for i in range(chrom_length):
            if similarity_vector[i] == 1:
                child_one.genes[i] = 1
                child_two.genes[i] = 1
            elif similarity_vector[i] == 0:
                child_one.genes[i] = 0
                child_two.genes[i] = 0
            else:  #the similarity_vector[i] is -1
                w = np.random.uniform(low=0, high=1) #for the first child
                if w < 0.5:
                    child_one.genes[i] = 1
                else:
                    child_one.genes[i] = 0

                w = np.random.uniform(low=0, high=1) #for the second child
                if w < 0.5:
                    child_two.genes[i] = 1
                else:
                    child_two.genes[i] = 0

    else:  # if pc is less than random number then don't make any change
        child_one = deepcopy(parent_one)
        child_two = deepcopy(parent_two)

    # updating fitnesses for children
    child_one.fitness = fitness_function(child_one.genes)
    child_two.fitness = fitness_function(child_two.genes)

    return child_one, child_two


chrom0 = np.array([1, 1, 1, 0, 1, 0, 0, 1, 0])
chrom1 = np.array([1, 0, 0, 0, 1, 0, 1, 1, 0])
parent_one = Chromosome(chrom0, id_= 0, fitness = fitness_function(chrom0))  
parent_two = Chromosome(chrom1, id_= 1, fitness = fitness_function(chrom1))
CROSS = random_respectful_crossover(parent_one, parent_two, 1)

if __name__ == '__main__':
    print("\nChildren")
    print("=================================================")
    for index, _ in enumerate(CROSS):  #two children
        Chromosome.describe(CROSS[index])



Parents
ID=#0, Fitness=5, 
Genes=
[1 1 1 0 1 0 0 1 0]
ID=#1, Fitness=4, 
Genes=
[1 0 0 0 1 0 1 1 0]

Similarity-vector: [1, -1, -1, 0, 1, 0, -1, 1, 0]

Children
ID=#0, Fitness=4, 
Genes=
[1 1 0 0 1 0 0 1 0]
ID=#1, Fitness=3, 
Genes=
[1 0 0 0 1 0 0 1 0]


## <a id='masked'>Masked Crossover</a>

In masked crossover, we first copy the value of the genes from parent-one to child-one and from parent-two to child-two. In this crossover, we assume that each individual in the population has a `mask vector`. The values of the mask vector are calculated based on the individual's fitness. If the value of a mask vector in a position is 1 and the value of another mask vector in that position is 0, we say that the first mask vector is dominant in that position. If we consider `mask_vector1` as the mask vector of parent-one, and `mask_vector2` as the mask vector of parent-two, the offspring will be calculated as the following.
> 1. If <span style="color:#EE7766">**mask_vector2(i) = 1** and **mask_vector1(i) = 0**</span>, then **child_one(i) = parent_two(i)**.
> 2. If <span style="color:#6677EE">**mask_vector2(i) = 0** and **mask_vector1(i) = 1**</span>, then **child_two(i) = parent_one(i)**.

<figure>
    <center> <img src="./pics/Crossover/masked_crossover.png"  alt='missing' width="450"  ><center/>
<figure/>

In [19]:
import numpy as np
from copy import deepcopy

def masked_crossover(parent_one, parent_two, pc):
    """
    This function takes two parents, and performs Masked crossover on them.
    In this crossover, each of the parents have a masked vector, and this vector for an individual 
    is calculated based on the fitness of that individual within the population.
    In this method, we used vectors with random values for this purpose.    
    parent_one: First parent
    parent_two: Second parent
    pc: The probability of crossover
    """  

    print("\nParents")
    print("=================================================")
    Chromosome.describe(parent_one)
    Chromosome.describe(parent_two)
    
    chrom_length = Chromosome.get_chrom_length(parent_one)
    child_one = Chromosome(genes= np.array([0]*chrom_length),id_=0,fitness = 125.2)   
    child_two = Chromosome(genes= np.array([0]*chrom_length),id_=1,fitness = 125.2)
    
    mask_one = np.random.choice([0, 1], size=chrom_length, p=[0.5, 0.5]) #mask vector for parent_one
    mask_two = np.random.choice([0, 1], size=chrom_length, p=[0.5, 0.5]) #mask vector for parent_two
    print("\nmask vector for parent_one: ", mask_one)
    print("mask vector for parent_two: ", mask_two)

    if np.random.rand() < pc:  # if pc is greater than random number
        for i in range(chrom_length): #Copying parents into children 
            child_one.genes[i] = parent_one.genes[i]
            child_two.genes[i] = parent_two.genes[i]

        for i in range(chrom_length):
            if mask_one[i] == 0 and mask_two[i] == 1: #The parent_two is dominant
                child_one.genes[i] = parent_two.genes[i]
            if mask_one[i] == 1 and mask_two[i] == 0: #The parent_one is dominant
                child_two.genes[i] = parent_one.genes[i]
            
        
    else:  # if pc is less than random number then don't make any change
        child_one = deepcopy(parent_one)
        child_two = deepcopy(parent_two)

    # updating fitnesses for children
    child_one.fitness = fitness_function(child_one.genes)
    child_two.fitness = fitness_function(child_two.genes)

    return child_one, child_two



chrom0 = np.array([1, 1, 1, 0, 1, 0, 0, 1, 0])
chrom1 = np.array([1, 0, 0, 0, 1, 0, 1, 1, 0])
parent_one = Chromosome(chrom0, id_= 0, fitness = fitness_function(chrom0))  
parent_two = Chromosome(chrom1, id_= 1, fitness = fitness_function(chrom1))
CROSS = masked_crossover(parent_one, parent_two, 1)

if __name__ == '__main__':
    print("\nChildren")
    print("=================================================")
    for index, _ in enumerate(CROSS):  #two children
        Chromosome.describe(CROSS[index])




Parents
ID=#0, Fitness=5, 
Genes=
[1 1 1 0 1 0 0 1 0]
ID=#1, Fitness=4, 
Genes=
[1 0 0 0 1 0 1 1 0]

mask vector for parent_one:  [1 0 1 0 1 1 0 1 0]
mask vector for parent_two:  [1 1 1 0 1 0 1 1 0]

Children
ID=#0, Fitness=5, 
Genes=
[1 0 1 0 1 0 1 1 0]
ID=#1, Fitness=4, 
Genes=
[1 0 0 0 1 0 1 1 0]


## <a id='1-bit-adaptive'>1-bit Adaptive Crossover (1BX)</a>

In 1-bit adaptive crossover, we look at the value of the last gene in both of the parents. 

**(1)** If <span style="color:#EE7766">the last two bits in the parents were the same and equal to 0</span>, then we perform **uniform crossover** on the parents.<br>
**(2)** If <span style="color:#77EEAA">the last two bits in the parents were the same and equal to 1</span>, then we perform **two-point crossover** on the parents.<br>
**(3)** If <span style="color:#6767EE">the last two bits in the parents were different</span>, we create a uniform random real number w to choose which crossover to perform. If w is less than 0.5, then we perform **uniform crossover**; otherwise, we perform **two-point crossover**.


<figure>
    <center> <img src="./pics/Crossover/1bx.png"  alt='missing' width="450"  ><center/>
<figure/>

In [20]:
import numpy as np
from copy import deepcopy

def claculate_mask(chrom_size, px):
    """
    This method, performs the mask calculation for the Uniform crossover.
    chrom_size: The mask has the same size of the chromosome.
    px: A control parameter for the number of ones in the mask vector.
    """  
    mask = [0]*chrom_size
    for i in range(chrom_size):
        prob = np.random.rand()
        if prob <= px:
            mask[i]=1
    return mask

def uniform_crossover(parent_one, parent_two, pc):
    """
    This function takes two parents, and performs Uniform crossover on them. 
    parent_one: First parent
    parent_two: Second parent
    pc: The probability of crossover
    """  

    print("\nParents")
    print("=================================================")
    Chromosome.describe(parent_one)
    Chromosome.describe(parent_two)
    
    chrom_length = Chromosome.get_chrom_length(parent_one)
    child_one = Chromosome(genes= np.array([0]*chrom_length),id_=0,fitness = 125.2)   
    child_two = Chromosome(genes= np.array([0]*chrom_length),id_=1,fitness = 125.2)
    if np.random.rand() < pc:  # if pc is greater than random number
        mask = claculate_mask(chrom_length,np.random.uniform(low=0.2, high=0.85))
        for i in range(chrom_length):
            if mask[i]==1:
                child_one.genes[i] = parent_one.genes[i]
                child_two.genes[i] = parent_two.genes[i]
            else:
                child_one.genes[i] = parent_two.genes[i]
                child_two.genes[i] = parent_one.genes[i]
           
    else:  # if pc is less than random number then don't make any change
        child_one = deepcopy(parent_one)
        child_two = deepcopy(parent_two)
    return child_one, child_two


def one_bit_adaptive_crossover(parent_one, parent_two, pc):
    """
    This function takes two parents, and performs 1_Bit adaptive crossover on them. 
    The last gene of two parents are the same and both zero ==> Uniform crossover
    The last gene of two parents are the same and both one ==> Multi-point crossover  
    parent_one: First parent
    parent_two: Second parent
    pc: The probability of crossover
    """  
    
    chrom_length = Chromosome.get_chrom_length(parent_one)
    child_one = Chromosome(genes= np.array([-1]*chrom_length),id_=0,fitness = 125.2)   
    child_two = Chromosome(genes= np.array([-1]*chrom_length),id_=1,fitness = 125.2)
    
    if np.random.rand() < pc:  # if pc is greater than random number
        
        if parent_one.genes[-1] == 0 and parent_two.genes[-1] == 0:
            child_one, child_two = uniform_crossover(parent_one,parent_two,1)
        elif parent_one.genes[-1] == 1 and parent_two.genes[-1] == 1:
            child_one, child_two = multi_point_crossover(parent_one,parent_two,1) 
        else:
            w = np.random.uniform(low=0, high=1)
            if w < 0.5:
                child_one, child_two = uniform_crossover(parent_one,parent_two,1)
            else:
                child_one, child_two = multi_point_crossover(parent_one,parent_two,1) 
                
        
    else:  # if pc is less than random number then don't make any change
        child_one = deepcopy(parent_one)
        child_two = deepcopy(parent_two)
    
    # updating fitnesses for children
    child_one.fitness = fitness_function(child_one.genes)
    child_two.fitness = fitness_function(child_two.genes)

    return child_one, child_two



chrom0 = np.array([1, 1, 1, 0, 1, 0, 0, 1, 0])
chrom1 = np.array([1, 0, 0, 0, 1, 0, 1, 1, 0])
parent_one = Chromosome(chrom0, id_= 0, fitness = fitness_function(chrom0))  
parent_two = Chromosome(chrom1, id_= 1, fitness = fitness_function(chrom1))

CROSS = one_bit_adaptive_crossover(parent_one, parent_two, 1)

if __name__ == '__main__':
    print("\nChildren")
    print("=================================================")
    for index, _ in enumerate(CROSS):  #two children
        Chromosome.describe(CROSS[index])



Parents
ID=#0, Fitness=5, 
Genes=
[1 1 1 0 1 0 0 1 0]
ID=#1, Fitness=4, 
Genes=
[1 0 0 0 1 0 1 1 0]

Children
ID=#0, Fitness=3, 
Genes=
[1 0 0 0 1 0 0 1 0]
ID=#1, Fitness=6, 
Genes=
[1 1 1 0 1 0 1 1 0]


## <a id='modified-partially-mapped'>Modified Partially Mapped Crossover (MPMX) </a>

In modified partially mapped crossover, we first produce two randomly selected positions. We transfer every gene between these two points directly from parent-one to child-one and from parent-two to child-two. Then, we do the following:

(1). <span style="color:#998800">**For child-one**</span>, we fill the positions that can be filled (the elements that don't already exist in child-one) by directly transferring the genes in these positions from parent-two to child-one. After that, we shuffle the remaining elements and fill the empty positions in child-one with them. 

(2). <span style="color:#008877"> **For child-two**</span>, we fill the positions that can be filled by directly transferring the genes in those positions from parent-one to child-two. Then, we shuffle the remaining elements and fill the empty positions in child-two with them.

<figure>
    <center> <img src="./pics/Crossover/modified_partially_mapped_crossover.png"  alt='missing' width="700"  ><center/>
<figure/>

In [21]:
import numpy as np
from copy import deepcopy

def modified_partially_mapped_crossover(parent_one, parent_two, pc):
    """
    This function takes two parents, and performs Modified partially mapped crossover on them.  
    parent_one: First parent
    parent_two: Second parent
    pc: The probability of crossover
    """  
    
    print("\nParents")
    print("=================================================")
    Chromosome.describe(parent_one)
    Chromosome.describe(parent_two)
    
    chrom_length = Chromosome.get_chrom_length(parent_one)
    child_one = Chromosome(genes= np.array([-1]*chrom_length),id_=0,fitness = 125.2)   
    child_two = Chromosome(genes= np.array([-1]*chrom_length),id_=1,fitness = 125.2)
    points = sorted_random_numbers(2,chrom_length)
    # points[0] = 2
    # points[1] = 6
    seen_list1 = [] #Elements that are transfered to child_one
    seen_list2 = [] #Elements that are transfered to child_two
    not_seen_list1 = [] #Elements that are not transfered to child_one
    not_seen_list2 = [] #Elements that are not transfered to child_two
    
    if np.random.rand() < pc:  # if pc is greater than random number
        for i in range(points[0], points[1]):
            child_one.genes[i] = parent_one.genes[i]
            seen_list1.append(parent_one.genes[i])
            child_two.genes[i] = parent_two.genes[i]
            seen_list2.append(parent_two.genes[i])

        for i in range(chrom_length):
            if i < points[0] or i >= points[1]:
                if parent_two.genes[i] not in seen_list1:
                    child_one.genes[i] = parent_two.genes[i]
                    seen_list1.append(parent_two.genes[i])
                if parent_one.genes[i] not in seen_list2:
                    child_two.genes[i] = parent_one.genes[i]
                    seen_list2.append(parent_one.genes[i])
        
        for i in range(chrom_length):
            if parent_one.genes[i] not in seen_list1:
                not_seen_list1.append(parent_one.genes[i])
            if parent_one.genes[i] not in seen_list2:
                not_seen_list2.append(parent_one.genes[i])

        np.random.shuffle(not_seen_list1) #shuffling the elements that are not present in children 
        np.random.shuffle(not_seen_list2)
        k = 0
        m = 0
        for i in range(chrom_length):
            if child_one.genes[i] == -1:
                child_one.genes[i] = not_seen_list1[k]
                k+=1
            if child_two.genes[i] == -1:
                child_two.genes[i] = not_seen_list2[m]
                m+=1                      
        
    else:  # if pc is less than random number then don't make any change
        child_one = deepcopy(parent_one)
        child_two = deepcopy(parent_two)

    # updating fitnesses for children
    child_one.fitness = fitness_function(child_one.genes)
    child_two.fitness = fitness_function(child_two.genes)

    return child_one, child_two


chrom0 = np.array([0, 8, 4, 5, 6, 7, 1, 2, 3, 9])
chrom1 = np.array([6, 7, 1, 2, 4, 8, 3, 5, 9, 0])
parent_one = Chromosome(chrom0, id_= 0, fitness = fitness_function(chrom0))  
parent_two = Chromosome(chrom1, id_= 1, fitness = fitness_function(chrom1))
CROSS = modified_partially_mapped_crossover(parent_one, parent_two, 1)

if __name__ == '__main__':
    print("\nChildren")
    print("=================================================")
    for index, _ in enumerate(CROSS):  #two children
        Chromosome.describe(CROSS[index])




Parents
ID=#0, Fitness=5, 
Genes=
[0 8 4 5 6 7 1 2 3 9]
ID=#1, Fitness=5, 
Genes=
[6 7 1 2 4 8 3 5 9 0]

Children
ID=#0, Fitness=5, 
Genes=
[2 8 4 5 6 7 3 1 9 0]
ID=#1, Fitness=5, 
Genes=
[0 7 1 2 4 8 5 6 3 9]


## <a id='order-based'>Order-Based Crossover (OBX)</a>

In order-based crossover, we choose **three random genes** from parent-two and we find their place in parent-one. We keep these places in child-one empty and fill every other position in child-one by transferring the genes from parent-one directly to them. Then, we loop over parent-two and transfer to child-one the genes that are not already transferred to it. 

For child two, we already saved the chosen places in parent-one, so we keep the same places empty in child-two and fill every other position in child-two by transferring the genes from parent-two directly to them. Then, we loop over parent-one and transfer to child-two the genes that are not already transferred to it.

<figure>
    <center> <img src="./pics/Crossover/order_based_crossover.png"  alt='missing' width="450"  ><center/>
<figure/>

In [44]:
import numpy as np
from copy import deepcopy

def ordered_based_crossover(parent_one, parent_two, pc):
    """
    This function takes two parents, and performs Ordered based crossover on them.  
    parent_one: First parent
    parent_two: Second parent
    pc: The probability of crossover
    """  
    
    print("\nParents")
    print("=================================================")
    Chromosome.describe(parent_one)
    Chromosome.describe(parent_two)
    
    chrom_length = Chromosome.get_chrom_length(parent_one)
    child_one = Chromosome(genes= np.array([-1]*chrom_length),id_=0,fitness = 125.2)   
    child_two = Chromosome(genes= np.array([-1]*chrom_length),id_=1,fitness = 125.2)

    not_seen_list1 = [] #Elements that are not transfered to child_one
    not_seen_list2 = [] #Elements that are not transfered to child_two
    
    if np.random.rand() < pc:  # if pc is greater than random number
        points = sorted_random_numbers(3,chrom_length) 
        # points[0] = 3
        # points[1] = 4
        # points[2] = 5      
        for i in range(len(points)):
            not_seen_list1.append(parent_one.genes[points[i]])
            not_seen_list2.append(parent_two.genes[points[i]])
        
        for i in range(chrom_length): #transfering genes of parents to children except chosen positions
            if i not in points:
                child_one.genes[i] = parent_one.genes[i]
                child_two.genes[i] = parent_two.genes[i]

        k = 0
        for i in range(chrom_length):
            if parent_two.genes[i] in not_seen_list1: #elements in parent_two that are not in child_one
                child_one.genes[points[k]] = parent_two.genes[i]
                k+=1
        k = 0
        for i in range(chrom_length):
            if parent_one.genes[i] in not_seen_list2: #elements in parent_one that are not in child_two
                child_two.genes[points[k]] = parent_one.genes[i]
                k+=1

                          
        
    else:  # if pc is less than random number then don't make any change
        child_one = deepcopy(parent_one)
        child_two = deepcopy(parent_two)

    # updating fitnesses for children
    child_one.fitness = fitness_function(child_one.genes)
    child_two.fitness = fitness_function(child_two.genes)

    return child_one, child_two


chrom0 = np.array([0, 8, 4, 5, 6, 7, 1, 2, 3, 9])
chrom1 = np.array([6, 7, 1, 2, 4, 8, 3, 5, 9, 0])
parent_one = Chromosome(chrom0, id_= 0, fitness = fitness_function(chrom0))  
parent_two = Chromosome(chrom1, id_= 1, fitness = fitness_function(chrom1))
CROSS = ordered_based_crossover(parent_one, parent_two, 1)

if __name__ == '__main__':
    print("\nChildren")
    print("=================================================")
    for index, _ in enumerate(CROSS):  #two children
        Chromosome.describe(CROSS[index])





Parents
ID=#0, Fitness=5, 
Genes=
[0 8 4 5 6 7 1 2 3 9]
ID=#1, Fitness=5, 
Genes=
[6 7 1 2 4 8 3 5 9 0]

Children
ID=#0, Fitness=5, 
Genes=
[0 8 2 5 6 7 1 4 3 9]
ID=#1, Fitness=5, 
Genes=
[6 7 5 2 4 8 3 1 9 0]


## <a id='position-based'>Position-Based Crossover (POS)</a>

In position-based crossover, we choose several random genes from parent-two and directly transfer them to child-one. Then, we fill the empty positions in child one, by looping over parent-one and transferring all the genes that are not already transferred to child-one from parent-two. 

For child-two, we transfer the genes in the positions that were randomly chosen earlier, from parent-one directly to child-two. Then we loop over the parent-two and transfer all the genes that are not already transferred to child-two from parent-one.

<figure>
    <center> <img src="./pics/Crossover/position_based_crossover.png"  alt='missing' width="450"  ><center/>
<figure/>

In [23]:
import numpy as np
from copy import deepcopy

def position_based_crossover(parent_one, parent_two, pc):
    """
    This function takes two parents, and performs Position based crossover on them.  
    parent_one: First parent
    parent_two: Second parent
    pc: The probability of crossover
    """  
    
    print("\nParents")
    print("=================================================")
    Chromosome.describe(parent_one)
    Chromosome.describe(parent_two)
    
    chrom_length = Chromosome.get_chrom_length(parent_one)
    child_one = Chromosome(genes= np.array([-1]*chrom_length),id_=0,fitness = 125.2)   
    child_two = Chromosome(genes= np.array([-1]*chrom_length),id_=1,fitness = 125.2)

    seen_list1 = [] #Elements that are transfered to child_one
    seen_list2 = [] #Elements that are transfered to child_two
    
    if np.random.rand() < pc:  # if pc is greater than random number
        "points in the begining, is the positions that are to be transfered from parent_two to child_one."
        "Also the positions that are to be transfered from parent_one to child_two."
        points = sorted_random_numbers(3,chrom_length)  
        # points[0] = 1
        # points[1] = 2
        # points[2] = 5      
        for i in range(len(points)):
            seen_list1.append(parent_two.genes[points[i]])
            seen_list2.append(parent_one.genes[points[i]])
        
        for i in range(chrom_length): #transfering genes of parents to children in the chosen positions
            if i in points:
                child_one.genes[i] = parent_two.genes[i]
                child_two.genes[i] = parent_one.genes[i]

        points = [] #positions in the children that are not already filled
        for i in range(chrom_length):
            if child_one.genes[i] == -1:
                points.append(i)
        
        k = 0
        for i in range(chrom_length):
            if parent_one.genes[i] not in seen_list1: #elements in parent_one that are not in child_one
                child_one.genes[points[k]] = parent_one.genes[i]
                k+=1
        
        k = 0
        for i in range(chrom_length):
            if parent_two.genes[i] not in seen_list2: #elements in parent_two that are not in child_two
                child_two.genes[points[k]] = parent_two.genes[i]
                k+=1
                           
    else:  # if pc is less than random number then don't make any change
        child_one = deepcopy(parent_one)
        child_two = deepcopy(parent_two)

    # updating fitnesses for children
    child_one.fitness = fitness_function(child_one.genes)
    child_two.fitness = fitness_function(child_two.genes)

    return child_one, child_two


chrom0 = np.array([0, 8, 4, 5, 6, 7, 1, 2, 3, 9])
chrom1 = np.array([6, 7, 1, 2, 4, 8, 3, 5, 9, 0])
parent_one = Chromosome(chrom0, id_= 0, fitness = fitness_function(chrom0))  
parent_two = Chromosome(chrom1, id_= 1, fitness = fitness_function(chrom1))

CROSS = position_based_crossover(parent_one, parent_two, 1)

if __name__ == '__main__':
    print("\nChildren")
    print("=================================================")
    for index, _ in enumerate(CROSS):  #two children
        Chromosome.describe(CROSS[index])




Parents
ID=#0, Fitness=5, 
Genes=
[0 8 4 5 6 7 1 2 3 9]
ID=#1, Fitness=5, 
Genes=
[6 7 1 2 4 8 3 5 9 0]

Children
ID=#0, Fitness=5, 
Genes=
[8 7 5 6 4 1 2 3 9 0]
ID=#1, Fitness=5, 
Genes=
[7 8 1 2 6 4 3 5 0 9]


## <a id='voting-recombination'>Voting Recombination Crossover (VR)</a>

In voting recombination crossover, we have **several parents** and a positive integer as a `threshold` which is less than or equal to the number of parents. If the value of a gene in a position has been the same in a number of parents equal to or greater than the threshold, then the value of the gene in that position for the offspring will become the same as that value. The remaining elements are shuffled and transferred to the offspring. <br>
In the following example, we have four parents and the threshold is 3.

<figure>
    <center> <img src="./pics/Crossover/voting_recombination_crossover.png"  alt='missing' width="600"  ><center/>
<figure/>

In [24]:
import numpy as np
from copy import deepcopy
from collections import Counter

def most_frequent(List):
    """
    Returns the element with most occurrence in a list
    """ 
    occurence_count = Counter(List) 
    return occurence_count.most_common(1)[0][0] 


def voting_recombination_crossover(parents, pc):
    """
    This function takes several parents, and performs Voting recombination crossover on them. 
    In this method, threshold is the number of repeatition for an element in one position, for 
    the child would be the same element, in that position. 
    parent_one: First parent
    parent_two: Second parent
    pc: The probability of crossover
    """  
    chrom_length = Chromosome.get_chrom_length(parents[0])
    
    print("\nParents")
    print("=================================================")
    for i in range(len(parents)):
        Chromosome.describe(parents[i])

    child_one = Chromosome(genes= np.array([-1]*chrom_length),id_=0,fitness = 125.2)   
    child_two = Chromosome(genes= np.array([-1]*chrom_length),id_=1,fitness = 125.2)
    threshold = 3
    seen_list = [] #Elements that are transfered to child
    not_seen_list = [] #Elements that are not transfered to child
    
    if np.random.rand() < pc:  # if pc is greater than random number
        for i in range(chrom_length):
            column = []
            for j in range(len(parents)):
                column.append(parents[j].genes[i])

            vote = most_frequent(column)
            votes_num = column.count(vote)
            if votes_num >= threshold:
                child_one.genes[i] = vote
                child_two.genes[i] = vote
                seen_list.append(vote)
        
        for i in range(chrom_length):
            if parents[0].genes[i] not in seen_list:
                not_seen_list.append(parents[0].genes[i])

        np.random.shuffle(not_seen_list)     
        k = 0
        for i in range(chrom_length): #filling empty elements of first child
            if child_one.genes[i] == -1:
                child_one.genes[i] = not_seen_list[k]
                k+=1

        np.random.shuffle(not_seen_list)     
        k = 0
        for i in range(chrom_length): #filling empty elements of second child
            if child_two.genes[i] == -1:
                child_two.genes[i] = not_seen_list[k]
                k+=1
     
    else:  # if pc is less than random number then the children will be one of the parents involved
        child_one = deepcopy(parents[np.random.randint(0, len(parents))])
        child_two = deepcopy(parents[np.random.randint(0, len(parents))])

    # updating fitnesses for children
    child_one.fitness = fitness_function(child_one.genes)
    child_two.fitness = fitness_function(child_two.genes)

    return child_one, child_two

# parents
parents = []
genes0 = np.array([1, 4, 3, 5, 2, 6, 0])
genes1 = np.array([1, 2, 0, 3, 5, 6, 4])
genes2 = np.array([3, 2, 1, 5, 4, 0, 6])
genes3 = np.array([1, 2, 3, 4, 0, 6, 5])
genes4 = np.array([0, 4, 2, 1, 6, 5, 3])

parents.append(Chromosome(genes= np.array(genes0), id_=0, fitness = fitness_function(genes0)))
parents.append(Chromosome(genes= np.array(genes1), id_=1, fitness = fitness_function(genes1)))
parents.append(Chromosome(genes= np.array(genes2), id_=2, fitness = fitness_function(genes2)))
parents.append(Chromosome(genes= np.array(genes3), id_=3, fitness = fitness_function(genes3)))
parents.append(Chromosome(genes= np.array(genes4), id_=4, fitness = fitness_function(genes4)))

CROSS = voting_recombination_crossover(parents, 1)

if __name__ == '__main__':
    print("\nChildren")
    print("=================================================")
    for index, _ in enumerate(CROSS):  #two children
        Chromosome.describe(CROSS[index])




Parents
ID=#0, Fitness=5, 
Genes=
[1 4 3 5 2 6 0]
ID=#1, Fitness=5, 
Genes=
[1 2 0 3 5 6 4]
ID=#2, Fitness=5, 
Genes=
[3 2 1 5 4 0 6]
ID=#3, Fitness=5, 
Genes=
[1 2 3 4 0 6 5]
ID=#4, Fitness=5, 
Genes=
[0 4 2 1 6 5 3]

Children
ID=#0, Fitness=5, 
Genes=
[1 2 5 4 3 6 0]
ID=#1, Fitness=5, 
Genes=
[1 2 4 5 3 6 0]


## <a id='maximal-preservation'>Maximal Preservation Crossover (MPX)</a>

In maximal preservation crossover, we select **a randomly appropriately sized substring** from parent-one and transfer the genes in that substring directly from parent-one to child-one. The transferred elements are deleted from parent-two and the remaining elements are transferred from parent-two to child-one. 

For child-two, in the same way, we select a randomly appropriately sized substring from parent-two and transfer the genes in that substring directly from parent-two to child-one. The transferred elements are deleted from parent-one and the remaining elements are transferred from parent-one to child-two.

<figure>
    <center> <img src="./pics/Crossover/mpx.png"  alt='missing' width="450"  ><center/>
<figure/>

In [25]:
import numpy as np
from copy import deepcopy

def maximal_preservation_crossover(parent_one, parent_two, pc):
    """
    This function takes two parents, and performs Maximal perservation crossover on them.  
    parent_one: First parent
    parent_two: Second parent
    pc: The probability of crossover
    """  

    print("\nParents")
    print("=================================================")
    Chromosome.describe(parent_one)
    Chromosome.describe(parent_two)
    
    chrom_length = Chromosome.get_chrom_length(parent_one)
    child_one = Chromosome(genes= np.array([-1]*chrom_length),id_=0,fitness = 125.2)   
    child_two = Chromosome(genes= np.array([-1]*chrom_length),id_=1,fitness = 125.2)

    
    seen_list1 = [] #Elements that are transfered to child_one
    seen_list2 = [] #Elements that are transfered to child_two
    
    if np.random.rand() < pc:  # if pc is greater than random number
        #********************************************************* Filling Child_one
        points = sorted_random_numbers(2, chrom_length) #for choosing a substring
        # points[0] = 1
        # points[1] = 5
        for i in range(points[0], points[1]): #transfering the substring from parent_one to child_one
            child_one.genes[i] = parent_one.genes[i]
            seen_list1.append(parent_one.genes[i])
        
        points = [] #finding empty spots in child_one
        for i in range(chrom_length):
            if child_one.genes[i] == -1:
                points.append(i)
        
        k = 0
        for i in range(chrom_length):
            if parent_two.genes[i] not in seen_list1:
                child_one.genes[points[k]] = parent_two.genes[i]
                k+=1
        #********************************************************* Filling Child_two
        points = sorted_random_numbers(2, chrom_length) #for choosing a substring
        # points[0] = 2
        # points[1] = 6
        for i in range(points[0], points[1]): #transfering the substring from parent_one to child_one
            child_two.genes[i] = parent_two.genes[i]
            seen_list2.append(parent_two.genes[i])
        
        points = [] #finding empty spots in child_two
        for i in range(chrom_length):
            if child_two.genes[i] == -1:
                points.append(i)
        
        k = 0
        for i in range(chrom_length):
            if parent_one.genes[i] not in seen_list2:
                child_two.genes[points[k]] = parent_one.genes[i]
                k+=1
                           
    else:  # if pc is less than random number then don't make any change
        child_one = deepcopy(parent_one)
        child_two = deepcopy(parent_two)
    
    # updating fitnesses for children
    child_one.fitness = fitness_function(child_one.genes)
    child_two.fitness = fitness_function(child_two.genes)

    return child_one, child_two



chrom0 = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
chrom1 = np.array([1, 4, 3, 2, 6, 7, 5, 9, 8, 0])
parent_one = Chromosome(chrom0, id_= 0, fitness = fitness_function(chrom0))  
parent_two = Chromosome(chrom1, id_= 1, fitness = fitness_function(chrom1))
CROSS = maximal_preservation_crossover(parent_one, parent_two, 1)

if __name__ == '__main__':
    print("\nChildren")
    print("=================================================")
    for index, _ in enumerate(CROSS):  #two children
        Chromosome.describe(CROSS[index])




Parents
ID=#0, Fitness=5, 
Genes=
[0 1 2 3 4 5 6 7 8 9]
ID=#1, Fitness=5, 
Genes=
[1 4 3 2 6 7 5 9 8 0]

Children
ID=#0, Fitness=5, 
Genes=
[1 4 3 2 6 5 9 7 8 0]
ID=#1, Fitness=5, 
Genes=
[0 1 3 2 6 7 4 5 8 9]


## <a id='position'>Position Crossover (PX)</a>

In position crossover, several positions are randomly selected in parent-one and transferred to child-one. The transferred elements are deleted from parents-two and by a loop on the remaining elements of parents-two, the elements that are not already transferred to child-one are transferred to it.

For child two, in the same way, several positions are randomly selected in parent-two and transferred to child-two. The transferred elements are deleted from parent-one and by a loop on the remaining elements of parents-one, the elements that are not already transferred to child-two are transferred to it. 

<span style="color:#EE9977">The main difference between position crossover and maximal preservation crossover is that in the position crossover the randomly selected elements are not necessarily in a sequence.</span>

<figure>
    <center> <img src="./pics/Crossover/position_crossover.png"  alt='missing' width="450"  ><center/>
<figure/>

In [26]:
import numpy as np
from copy import deepcopy

def position_crossover(parent_one, parent_two, pc):
    """
    This function takes two parents, and performs Position crossover on them.
    In this method, we assumong the length of chromosomes are more than 5.
    The main difference of this crossover with Maximal-preservation crossover, is that the points 
    that are being transfered directly to children, are not necessarily consequent.   
    parent_one: First parent
    parent_two: Second parent
    pc: The probability of crossover
    """  
  
    print("\nParents")
    print("=================================================")
    Chromosome.describe(parent_one)
    Chromosome.describe(parent_two)
    
    chrom_length = Chromosome.get_chrom_length(parent_one)
    child_one = Chromosome(genes= np.array([-1]*chrom_length),id_=0,fitness = 125.2)   
    child_two = Chromosome(genes= np.array([-1]*chrom_length),id_=1,fitness = 125.2)

    
    seen_list1 = [] #Elements that are transfered to child_one
    seen_list2 = [] #Elements that are transfered to child_two
    
    if np.random.rand() < pc:  # if pc is greater than random number
        #********************************************************* Filling Child_one
        point_nums = np.random.randint(2,chrom_length-2) #number of points
        points = sorted_random_numbers(point_nums, chrom_length) #for choosing several points

        for i in range(chrom_length):
            if i in points:
                child_one.genes[i] = parent_one.genes[i]
                seen_list1.append(parent_one.genes[i])
        
        points = [] #finding empty spots in child_one
        for i in range(chrom_length):
            if child_one.genes[i] == -1:
                points.append(i)
        k = 0
        for i in range(chrom_length):
            if parent_two.genes[i] not in seen_list1:
                child_one.genes[points[k]] = parent_two.genes[i]
                k+=1
        #********************************************************* Filling Child_two
        point_nums = np.random.randint(2,chrom_length-2) #number of points
        points = sorted_random_numbers(point_nums, chrom_length) #for choosing several points

        for i in range(chrom_length):
            if i in points:
                child_two.genes[i] = parent_two.genes[i]
                seen_list2.append(parent_two.genes[i])
        
        points = [] #finding empty spots in child_two
        for i in range(chrom_length):
            if child_two.genes[i] == -1:
                points.append(i)
        k = 0
        for i in range(chrom_length):
            if parent_one.genes[i] not in seen_list2:
                child_two.genes[points[k]] = parent_one.genes[i]
                k+=1
                           
    else:  # if pc is less than random number then don't make any change
        child_one = deepcopy(parent_one)
        child_two = deepcopy(parent_two)

    # updating fitnesses for children
    child_one.fitness = fitness_function(child_one.genes)
    child_two.fitness = fitness_function(child_two.genes)

    return child_one, child_two



chrom0 = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
chrom1 = np.array([1, 4, 3, 2, 6, 7, 5, 9, 8, 0])
parent_one = Chromosome(chrom0, id_= 0, fitness = fitness_function(chrom0))  
parent_two = Chromosome(chrom1, id_= 1, fitness = fitness_function(chrom1))
CROSS = position_crossover(parent_one, parent_two, 1)

if __name__ == '__main__':
    print("\nChildren")
    print("=================================================")
    for index, _ in enumerate(CROSS):  #two children
        Chromosome.describe(CROSS[index])



Parents
ID=#0, Fitness=5, 
Genes=
[0 1 2 3 4 5 6 7 8 9]
ID=#1, Fitness=5, 
Genes=
[1 4 3 2 6 7 5 9 8 0]

Children
ID=#0, Fitness=5, 
Genes=
[4 1 3 2 6 5 8 7 0 9]
ID=#1, Fitness=5, 
Genes=
[0 1 3 2 4 7 5 9 6 8]


## <a id='homologous'>Homologous Crossover (HX)</a>

In homologous crossover, we divide the parents into several substrings and only those substrings with enough similarity are allowed to be performed multi-point crossover on them. To calculate the similarity, we define a vector named `similarity vector` and a variable named `threshold`. The similarity vector is calculated by performing exclusive or (XOR) on the genes of the parents.

A `1` value in the similarity vector means that the value of the genes in that place is different and the `0` value means that the value of the genes in that place is the same. If the number of 1s in a substring in the similarity vector is less than the threshold, it means that substring has enough similarity to be allowed for multi-point crossover.

<figure>
    <center> <img src="./pics/Crossover/homologous_crossover.png"  alt='missing' width="600"  ><center/>
<figure/>

In [27]:
import numpy as np
from copy import deepcopy

def homologous_crossover(parent_one, parent_two, pc):
    """
    This function takes two parents, and performs Homologous crossover on them. 
    parent_one: First parent
    parent_two: Second parent
    pc: The probability of crossover
    """  

    print("\nParents")
    print("=================================================")
    Chromosome.describe(parent_one)
    Chromosome.describe(parent_two)
    
    chrom_length = Chromosome.get_chrom_length(parent_one)
    child_one = Chromosome(genes= np.array([-1]*chrom_length),id_=0,fitness = 125.2)   
    child_two = Chromosome(genes= np.array([-1]*chrom_length),id_=1,fitness = 125.2)

    child_one.genes = parent_one.genes #copying parents to their children
    child_two.genes = parent_two.genes

  

    if np.random.rand() < pc:  # if pc is greater than random number
        point1 = np.random.randint(0, int(chrom_length/2) - 1)
        point2 = np.random.randint(int(chrom_length/2) + 1, chrom_length)
        sum1 = 0
        sum2 = 0
        sum3 = 0
        sums = [] #the differences in each part of parents
        yardstick = [] #parts of parents that the diversity is greater than threshold to participate in multi_point_crossover
        threshold = 2
        similarity_vector = []
        for i in range(chrom_length):
            similarity_vector.append(parent_one.genes[i] ^ parent_two.genes[i])
        # print(similarity_vector)
        for i in range(chrom_length):
            if i < point1:
                sum1 = sum1 + similarity_vector[i]
            elif i >= point1 and i < point2:
                sum2 = sum2 + similarity_vector[i]
            else:
                sum3 = sum3 + similarity_vector[i]

        sums.append(sum1)
        sums.append(sum2)
        sums.append(sum3)
        print(sums)
        for i in range(len(sums)): #adding the parts with enough diversity
            if sums[i] >= threshold:
                yardstick.append(True)
            else:
                yardstick.append(False)

        print(yardstick)
        a = []
        b = []
        sub_parent_one = Chromosome(genes= np.array([-1]*chrom_length),id_=0,fitness = 125.2)   
        sub_parent_two = Chromosome(genes= np.array([-1]*chrom_length),id_=1,fitness = 125.2)        

        #*************************************************************************** Eigth Different States
        if yardstick[0] == True and yardstick[1] == True and yardstick[2] == True: #State 1
            child_one, child_two = multi_point_crossover(parent_one, parent_two, 1)

        elif yardstick[0] == True and yardstick[1] == True and yardstick[2] == False: #State 2
            for i in range(0, point2):
                a.append(parent_one.genes[i])
                b.append(parent_two.genes[i])

            sub_parent_one.genes = a
            sub_parent_two.genes = b

            sub_parent_one, sub_parent_two = multi_point_crossover(sub_parent_one, sub_parent_two, 1)

            k = 0
            for i in range(0, point2):
                child_one.genes[i] = sub_parent_one.genes[k]
                child_two.genes[i] = sub_parent_two.genes[k]
                k+=1
            
        elif yardstick[0] == True and yardstick[1] == False and yardstick[2] == True: #State 3
            for i in range(0, point1):
                a.append(parent_one.genes[i])
                b.append(parent_two.genes[i])

            for i in range(point2, chrom_length):
                a.append(parent_one.genes[i])
                b.append(parent_two.genes[i])

            sub_parent_one.genes = a
            sub_parent_two.genes = b

            sub_parent_one, sub_parent_two = multi_point_crossover(sub_parent_one, sub_parent_two, 1)
            
            k = 0
            for i in range(0, point1):
                child_one.genes[i] = sub_parent_one.genes[k]
                child_two.genes[i] = sub_parent_two.genes[k]
                k+=1                  
            for i in range(point2, chrom_length):
                child_one.genes[i] = sub_parent_one.genes[k]
                child_two.genes[i] = sub_parent_two.genes[k]
                k+=1

        elif yardstick[0] == False and yardstick[1] == True and yardstick[2] == True: #State 4
            for i in range(point1, chrom_length):
                a.append(parent_one.genes[i])
                b.append(parent_two.genes[i])

            sub_parent_one.genes = a
            sub_parent_two.genes = b

            sub_parent_one, sub_parent_two = multi_point_crossover(sub_parent_one, sub_parent_two, 1)

            k = 0
            for i in range(point1, chrom_length):
                child_one.genes[i] = sub_parent_one.genes[k]
                child_two.genes[i] = sub_parent_two.genes[k]
                k+=1

        elif yardstick[0] == False and yardstick[1] == False and yardstick[2] == True: #State 5
            for i in range(point2, chrom_length):
                a.append(parent_one.genes[i])
                b.append(parent_two.genes[i])

            sub_parent_one.genes = a
            sub_parent_two.genes = b

            sub_parent_one, sub_parent_two = multi_point_crossover(sub_parent_one, sub_parent_two, 1)

            k = 0
            for i in range(point2, chrom_length):
                child_one.genes[i] = sub_parent_one.genes[k]
                child_two.genes[i] = sub_parent_two.genes[k]
                k+=1

        elif yardstick[0] == False and yardstick[1] == True and yardstick[2] == False: #State 6
            for i in range(point1, point2):
                a.append(parent_one.genes[i])
                b.append(parent_two.genes[i])

            sub_parent_one.genes = a
            sub_parent_two.genes = b

            sub_parent_one, sub_parent_two = multi_point_crossover(sub_parent_one, sub_parent_two, 1)

            k = 0
            for i in range(point1, point2):
                child_one.genes[i] = sub_parent_one.genes[k]
                child_two.genes[i] = sub_parent_two.genes[k]
                k+=1

        elif yardstick[0] == True and yardstick[1] == False and yardstick[2] == False: #State 7
            for i in range(0, point1):
                a.append(parent_one.genes[i])
                b.append(parent_two.genes[i])

            sub_parent_one.genes = a
            sub_parent_two.genes = b

            sub_parent_one, sub_parent_two = multi_point_crossover(sub_parent_one, sub_parent_two, 1)

            k = 0
            for i in range(0, point1):
                child_one.genes[i] = sub_parent_one.genes[k]
                child_two.genes[i] = sub_parent_two.genes[k]
                k+=1


        #**************************************************************************************************

    else:  # if pc is less than random number then don't make any change
        child_one = deepcopy(parent_one)
        child_two = deepcopy(parent_two)

    # updating fitnesses for children
    child_one.fitness = fitness_function(child_one.genes)
    child_two.fitness = fitness_function(child_two.genes)

    return child_one, child_two


chrom0 = np.array([0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1])
chrom1 = np.array([1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0])
parent_one = Chromosome(chrom0, id_= 0, fitness = fitness_function(chrom0))  
parent_two = Chromosome(chrom1, id_= 1, fitness = fitness_function(chrom1))
CROSS = homologous_crossover(parent_one, parent_two, 1)

if __name__ == '__main__':
    print("\nChildren")
    print("=================================================")
    for index, _ in enumerate(CROSS):  #two children
        Chromosome.describe(CROSS[index])



Parents
ID=#0, Fitness=6, 
Genes=
[0 1 0 0 0 1 0 1 1 1 1]
ID=#1, Fitness=5, 
Genes=
[1 0 1 1 0 1 0 1 0 0 0]
[1, 3, 3]
[False, True, True]

Parents
ID=#0, Fitness=125.2, 
Genes=
[1, 0, 0, 0, 1, 0, 1, 1, 1, 1]
ID=#1, Fitness=125.2, 
Genes=
[0, 1, 1, 0, 1, 0, 1, 0, 0, 0]

Children
ID=#0, Fitness=5, 
Genes=
[0 1 1 0 0 1 0 1 0 1 0]
ID=#1, Fitness=6, 
Genes=
[1 0 0 1 0 1 0 1 1 0 1]


## <a id='complete-subtour-exchange'>Complete Subtour Exchange Crossover (CSEX)</a>

In complete subtour exchange crossover, equal or symmetric (reversed) substrings are found in the two parents and several offspring are produced based on different modes of these substrings. Two fittest of these offspring are returned as the offspring.

<figure>
    <center> <img src="./pics/Crossover/complete_subtour_exchange_crossover.png"  alt='missing' width="450"  ><center/>
<figure/>

In [28]:
import numpy as np
from copy import deepcopy

def complete_subtour_exchange_crossover(parent_one, parent_two, pc):
    """
    This function takes two parents, and performs Complete subtour exchange crossover on them. 
    parent_one: First parent
    parent_two: Second parent
    pc: The probability of crossover
    """  
    
    print("\nParents")
    print("=================================================")
    Chromosome.describe(parent_one)
    Chromosome.describe(parent_two)
    
    chrom_length = Chromosome.get_chrom_length(parent_one)
    child_one = Chromosome(genes= np.array([-1]*chrom_length),id_=0,fitness = 125.2)   
    child_two = Chromosome(genes= np.array([-1]*chrom_length),id_=1,fitness = 125.2)

    if np.random.rand() < pc:  # if pc is greater than random number
        list1 = [] #for saving the positions of the subset, in parent_one
        list2 = [] #for saving the positions of the subset, in parent_two
        total1 = [] #for saving all the positions of the subsets, in parent_one
        total2 = [] #for saving all the positions of the subsets, in parent_two
        for i in range(chrom_length):
            gene_index = parent_two.genes.tolist().index(parent_one.genes[i])
            # print(gene_index)
            if gene_index + 1 < chrom_length and i + 1 < chrom_length:
                if parent_two.genes[gene_index + 1] == parent_one.genes[i + 1]:
                    k = i
                    list1.append(k)
                    list2.append(gene_index)
                    for j in range(gene_index, chrom_length): #forward loop
                        if parent_two.genes[j] == parent_one.genes[k]:
                            k+=1
                        else:
                            break
                    
                    list1.append(k)
                    list2.append(j + 1)
                    total1.append(list1)
                    total2.append(list2)
                    list1 = []
                    list2 = []
        for i in range(chrom_length):
            gene_index = parent_two.genes.tolist().index(parent_one.genes[i])
            # print(gene_index)
            list1 = []
            list2 = []
            if gene_index - 1 >= 0 and i + 1 < chrom_length:
                if parent_two.genes[gene_index - 1] == parent_one.genes[i + 1]:
                    k = i
                    list1.append(k)
                    list2.append(gene_index + 1)
                    for j in range(gene_index, -1, -1): #backward loop
                        if parent_two.genes[j] == parent_one.genes[k]:
                            k+=1
                        else:
                            break

                    list1.append(k)
                    list2.append(j + 1)
                    total1.append(list1)
                    total2.append(list2)
                    list1 = []
                    list2 = []

        for i in range(len(total2)): #correcting the indexes in total2
            if total2[i][0] > total2[i][1]:
                total2[i][0], total2[i][1] = total2[i][1], total2[i][0]
        
        #************************************************* Deleting redundant subsets
        seen_list = []
        indexes = []
        for i in range(len(total1)): 
            if total1[i][1] not in seen_list:
                seen_list.append(total1[i][1])
            else:
                indexes.append(i)

        # print(indexes)
        for i in range(len(indexes)):
            total1.remove(total1[indexes[i]])
            total2.remove(total2[indexes[i]])
        #************************************************************ Children with one reversed subset
        children = []
        for i in range(len(total1)):
            interval = total1[i]
            child = parent_one.genes.tolist()
            reversed_list = []
            for j in range(interval[0], interval[1]):
                reversed_list.append(child[j])
            reversed_list.reverse()
            k = 0
            for j in range(interval[0], interval[1]):
                child[j] = reversed_list[k]
                k+=1
            children.append(child)      

        for i in range(len(total2)):
            interval = total2[i]
            child = parent_two.genes.tolist()
            reversed_list = []
            for j in range(interval[0], interval[1]):
                reversed_list.append(child[j])
            reversed_list.reverse()
            k = 0
            for j in range(interval[0], interval[1]):
                child[j] = reversed_list[k]
                k+=1
            children.append(child) 
    #****************************************************************** Adding the other children   
        combinations = []
        for i in range(len(total1)):
            for j in range(i, len(total1)):
                if i != j:
                    combinations.append([i, j])
                    l = []
                    for k in range(i, j+1):
                        l.append(k)
                    combinations.append(l)

        b = list()
        for sublist in combinations:
            if sublist not in b:
                b.append(sublist)
        combinations = b

        for i in range(len(combinations)):
            combination = combinations[i]
            child = parent_one.genes.tolist()
            for j in range(len(combination)):
                pos = combination[j]
                interval = total1[pos]
                reversed_list = []
                for m in range(interval[0], interval[1]):
                    reversed_list.append(child[m])
                reversed_list.reverse()
                # print(reversed_list)
                k = 0
                for m in range(interval[0], interval[1]):
                    child[m] = reversed_list[k]
                    k+=1

            children.append(child)
         
        for i in range(len(combinations)):
            combination = combinations[i]
            child = parent_two.genes.tolist()
            for j in range(len(combination)):
                pos = combination[j]
                interval = total2[pos]
                reversed_list = []
                for m in range(interval[0], interval[1]):
                    reversed_list.append(child[m])
                reversed_list.reverse()
                # print(reversed_list)
                k = 0
                for m in range(interval[0], interval[1]):
                    child[m] = reversed_list[k]
                    k+=1

            children.append(child)

          
        print("\nChildren:", children) 
    #************************************************************************
        num1 = np.random.randint(0, len(children))
        num2 = np.random.randint(0, len(children))
        # making sure that two different children have been chosen
        while num2 == num1: 
            num2 = np.random.randint(0, len(children))
        child_one.genes = np.array(children[num1]) 
        child_two.genes = np.array(children[num2])
                                                                                          

    else:  # if pc is less than random number then don't make any change
        child_one = deepcopy(parent_one)
        child_two = deepcopy(parent_two)

    # updating fitnesses for children
    child_one.fitness = fitness_function(child_one.genes)
    child_two.fitness = fitness_function(child_two.genes)

    return child_one, child_two


chrom0 = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
chrom1 = np.array([4, 9, 7, 6, 5, 0, 8, 2, 1, 3])
parent_one = Chromosome(chrom0, id_= 0, fitness = fitness_function(chrom0))  
parent_two = Chromosome(chrom1, id_= 1, fitness = fitness_function(chrom1))
CROSS = complete_subtour_exchange_crossover(parent_one, parent_two, 1)

if __name__ == '__main__':
    print("\nChildren")
    print("=================================================")
    for index, _ in enumerate(CROSS):  #two children
        Chromosome.describe(CROSS[index])




Parents
ID=#0, Fitness=5, 
Genes=
[0 1 2 3 4 5 6 7 8 9]
ID=#1, Fitness=5, 
Genes=
[4 9 7 6 5 0 8 2 1 3]

Children: [[0, 2, 1, 3, 4, 5, 6, 7, 8, 9], [0, 1, 2, 3, 4, 7, 6, 5, 8, 9], [4, 9, 7, 6, 5, 0, 8, 1, 2, 3], [4, 9, 5, 6, 7, 0, 8, 2, 1, 3], [0, 2, 1, 3, 4, 7, 6, 5, 8, 9], [4, 9, 5, 6, 7, 0, 8, 1, 2, 3]]

Children
ID=#0, Fitness=5, 
Genes=
[4 9 5 6 7 0 8 2 1 3]
ID=#1, Fitness=5, 
Genes=
[0 2 1 3 4 7 6 5 8 9]


## <a id='edge-recombination'>Edge Recombination Crossover (ER)</a>

In edge recombination crossover, we do the following to fill child-one.

1. The first element of child-one comes from parent-one. Then, we look through the neighbors list and delete the first element of parent-one from all of the rows. Then, we look through the neighbors of the selected elements (first element of parent-one) and find the element with the least number of neighbors in the neighbors list.

2. If the least number of elements for more than one element is the same, then we choose one of them randomly.

3. If we reach an element in the neighbors list which has no neighbors, then we shuffle the remaining elements which are not already transferred to child-one and transfer them to it. 

We do the same thing for child-two with the difference that the first element of child-two will come from parent-two.

<figure>
    <center> <img src="./pics/Crossover/edge_recombination_crossover.png"  alt='missing' width="450"  ><center/>
<figure/>

In [29]:
import numpy as np
from copy import deepcopy

def edge_recombination_crossover(parent_one, parent_two, pc):
    """
    This function takes two parents, and performs Edge recombination crossover on them. 
    parent_one: First parent
    parent_two: Second parent
    pc: The probability of crossover
    """  

    print("\nParents")
    print("=================================================")
    Chromosome.describe(parent_one)
    Chromosome.describe(parent_two)
    
    chrom_length = Chromosome.get_chrom_length(parent_one)
    child_one = Chromosome(genes= np.array([-1]*chrom_length),id_=0,fitness = 125.2)   
    child_two = Chromosome(genes= np.array([-1]*chrom_length),id_=1,fitness = 125.2)
  

    if np.random.rand() < pc:  # if pc is greater than random number
        neighbour_list = []
        for _ in range(chrom_length):
            a = []
            neighbour_list.append(a)
    #******************************************************** Filling neighbour_list 
        parent_one_copy = parent_one.genes.tolist().copy()
        parent_one_copy = sorted(parent_one_copy) #sorting the array for neighbour_list
        # print(parent_one_copy) #sorted parent_one
           
        for i in range(chrom_length):
            pos = np.where(parent_one.genes == parent_one_copy[i])
            neighbour_list[i].append(parent_one.genes[pos[0][0] - 1])           
            if pos[0][0] < chrom_length - 1:
                neighbour_list[i].append(parent_one.genes[pos[0][0] + 1])
            else:
                neighbour_list[i].append(parent_one.genes[0])

            pos = np.where(parent_two.genes == parent_one_copy[i])
            neighbour_list[i].append(parent_two.genes[pos[0][0] - 1])           
            if pos[0][0] < chrom_length - 1:
                neighbour_list[i].append(parent_two.genes[pos[0][0] + 1])
            else:
                neighbour_list[i].append(parent_two.genes[0])

        for i in range(chrom_length):
            neighbour_list[i] = list(dict.fromkeys(neighbour_list[i]))
        # print(neighbour_list)  
    #***********************************************************************************       
        neighbour_list_copy = neighbour_list.copy()
        child1 = []
        child2 = []
        #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% Filling child1
        child1.append(parent_one.genes[0])
        for i in range(chrom_length - 1):
            node = child1[-1]
            for j in range(chrom_length):
                if node in neighbour_list_copy[j]:
                    neighbour_list_copy[j].remove(node)

            node_index = parent_one_copy.index(node)
            # print(node_index)
            neighbour_lengths = []
            for j in range(len(neighbour_list_copy[node_index])): #traversing through the node looking for its remaining neighbours searching for their lengths
                neighbour = neighbour_list_copy[node_index][j] 
                neighbour_index = parent_one_copy.index(neighbour) #finding the index of that neighbour in neighbour_list
                neighbour_length = len(neighbour_list_copy[neighbour_index]) #calculating the length of that neighbour
                neighbour_lengths.append(neighbour_length) #saving the length of that neighbour in the list neighbour_lengths
                
            # print(neighbour_list)
            #replacing the previous node with the new node which is the neighbour of the previous node with the least length.
            if not not neighbour_lengths: #if neighbour_lengths is not empty
                item = np.min(neighbour_lengths)
            else:
                # print("problem")
                not_seen_list = []
                for k in range(chrom_length):
                    if parent_one.genes[k] not in child1:
                        not_seen_list.append(parent_one.genes[k])
                
                np.random.shuffle(not_seen_list)
                for k in range(len(not_seen_list)):
                    child1.append(not_seen_list[k])

                child_one.genes = np.array(child1)
                break
                
            count = 0
            for j in range(len(neighbour_lengths)): #checking whether the minimum is duplicated
                if neighbour_lengths[j] == item:
                    count+=1

            if count == 1: #if minimum is unique, then append the node with minimum length
                node = neighbour_list_copy[node_index][neighbour_lengths.index(np.min(neighbour_lengths))] 
            else: #if there are more than one neighbour with the least size, pick one of them randomly.
                duplicate_elements_indexes = []
                for j in range(len(neighbour_lengths)): #adding the indices of the duplicated element to the list duplicate_elements_indexes
                    if neighbour_lengths[j] == item:
                        duplicate_elements_indexes.append(j)
                node = neighbour_list_copy[node_index][np.random.choice(duplicate_elements_indexes)]


            child1.append(node) #appending the new node to child1
            

    #******************************************************** Filling neighbour_list          
        for i in range(chrom_length):
            pos = np.where(parent_one.genes == parent_one_copy[i])
            neighbour_list[i].append(parent_one.genes[pos[0][0] - 1])           
            if pos[0][0] < chrom_length - 1:
                neighbour_list[i].append(parent_one.genes[pos[0][0] + 1])
            else:
                neighbour_list[i].append(parent_one.genes[0])

            pos = np.where(parent_two.genes == parent_one_copy[i])
            neighbour_list[i].append(parent_two.genes[pos[0][0] - 1])           
            if pos[0][0] < chrom_length - 1:
                neighbour_list[i].append(parent_two.genes[pos[0][0] + 1])
            else:
                neighbour_list[i].append(parent_two.genes[0])

        for i in range(chrom_length):
            neighbour_list[i] = list(dict.fromkeys(neighbour_list[i]))
        # print(neighbour_list)  
    #*********************************************************************************** 
        #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% Filling child2
        neighbour_list_copy = neighbour_list.copy()
        # print(neighbour_list_copy)
        child2.append(parent_two.genes[0])
        for i in range(chrom_length - 1):
            node = child2[-1]
            for j in range(chrom_length):
                if node in neighbour_list_copy[j]:
                    neighbour_list_copy[j].remove(node)

            node_index = parent_one_copy.index(node)
            # print(node_index)
            neighbour_lengths = []
            for j in range(len(neighbour_list_copy[node_index])): #traversing through the node looking for its remaining neighbours searching for their lengths
                neighbour = neighbour_list_copy[node_index][j] 
                neighbour_index = parent_one_copy.index(neighbour) #finding the index of that neighbour in neighbour_list
                neighbour_length = len(neighbour_list_copy[neighbour_index]) #calculating the length of that neighbour
                neighbour_lengths.append(neighbour_length) #saving the length of that neighbour in the list neighbour_lengths
                
            # print(neighbour_lengths)
            #replacing the previous node with the new node which is the neighbour of the previous node with the least length.
            if not not neighbour_lengths: #if neighbour_lengths is not empty
                item = np.min(neighbour_lengths)
            else:
                # print("problem")
                not_seen_list = []
                for k in range(chrom_length):
                    if parent_one.genes[k] not in child2:
                        not_seen_list.append(parent_one.genes[k])
                
                np.random.shuffle(not_seen_list)
                for k in range(len(not_seen_list)):                   
                    child2.append(not_seen_list[k])
                
                child_two.genes = np.array(child2)
                break
                           
            count = 0
            for j in range(len(neighbour_lengths)): #checking whether the minimum is duplicated
                if neighbour_lengths[j] == item:
                    count+=1

            if count == 1: #if minimum is unique, then append the node with minimum length
                node = neighbour_list_copy[node_index][neighbour_lengths.index(np.min(neighbour_lengths))] 
            else: #if there are more than one neighbour with the least size, pick one of them randomly.
                duplicate_elements_indexes = []
                for j in range(len(neighbour_lengths)): #adding the indices of the duplicated element to the list duplicate_elements_indexes
                    if neighbour_lengths[j] == item:
                        duplicate_elements_indexes.append(j)
                node = neighbour_list_copy[node_index][np.random.choice(duplicate_elements_indexes)]


            # print(count)
            child2.append(node) #appending the new node to child1
            child_one.genes = np.array(child1)
            child_two.genes = np.array(child2)
            
    else:  # if pc is less than random number then don't make any change
        child_one = deepcopy(parent_one)
        child_two = deepcopy(parent_two)

    # updating fitnesses for children
    child_one.fitness = fitness_function(child_one.genes)
    child_two.fitness = fitness_function(child_two.genes)

    return child_one, child_two


chrom0 = np.array([5, 1, 7, 8, 4, 9, 6, 2, 3, 0])
chrom1 = np.array([3, 6, 2, 0, 5, 1, 9, 8, 4, 7])
parent_one = Chromosome(chrom0, id_= 0, fitness = fitness_function(chrom0))  
parent_two = Chromosome(chrom1, id_= 1, fitness = fitness_function(chrom1))
CROSS = edge_recombination_crossover(parent_one, parent_two, 1)

if __name__ == '__main__':
    print("\nChildren")
    print("=================================================")
    for index, _ in enumerate(CROSS):  #two children
        Chromosome.describe(CROSS[index])




Parents
ID=#0, Fitness=5, 
Genes=
[5 1 7 8 4 9 6 2 3 0]
ID=#1, Fitness=5, 
Genes=
[3 6 2 0 5 1 9 8 4 7]

Children
ID=#0, Fitness=5, 
Genes=
[5 0 2 3 6 9 1 7 4 8]
ID=#1, Fitness=5, 
Genes=
[3 0 2 6 9 1 5 7 8 4]


## <a id='alternate-edges'>Alternate Edges Crossover (AEX)</a>

The Alternate Edges Crossover (AEX) operator works by alternating between the edges of the two parents to create a new child.To operate alternate edges crossover on two selected parents, we first need to introduce a concept named `arc`. An `arc` is an edge in a parent solution (any pair of elements that are in a sequence). For instance, the arcs for parent-one is as follows: <br>
`5-1`, `1-7`, `7-8`, `8-4`, `4-9`, `9-6`, `6-2`, `2-3`, `3-0`, `0-5` 

Similarly, the arcs or edges in parent-two are: <br>
`3-6`, `6-2`, `2-0`, `0-5`, `5-1`, `1-9`, `9-8`, `8-4`, `4-7`, `7-3`

The AEX operator works by alternating between the edges of the two parents to create a new child. Now, we discuss how alternate edges crossover works with an example. Consider the following example where `parent-one = [5 1 7 8 4 9 6 2 3 0]` and `parent-two = [3 6 2 0 5 1 9 8 4 7]`.

For child-one, we start by the first arc in parent-one which is `5-1`. We find the position of the element `1` in parent-two and notice that the next arc in the sequence is `1-9`. Again, we look for element `9` in parent-one and we find out that the next arc in the sequence is `9-6`. We keep alternating between the parents, until we reach a cycle (where the next element in the sequence is already seen). If we find the element `3` in parent-two, the next element in the sequence would be `6` which is already seen. So, up until now, we have the following for child-one: <br>
`child-one = [5 1 9 6 2 3 -1 -1 -1 -1]` <br>
where -1s denote the positions that are yet to be filled. We reached a cycle in parent-two, so we will randomly choose a value that is not already seen from the other parent which in this case is parent-one. If we choose `7` in parent-one, the corresponding arc would be `7-8` in parent-one, which then gives us the arc `8-4` in parent-two, and then the arc `4-9` in parent-one. Element `9` is already seen, therefore, we have another cycle. Thus far, we have the following for child-one: <br>
`child-one = [5 1 9 6 2 3 7 8 4 -1]` <br>
The only unseen element in parent-two is `0`. So, the final result for child-one would be: <br>
`child-one = [5 1 9 6 2 3 7 8 4 0]` <br>

We have a similar approach to calculate the genes of child-two, but this time, we start off by the arc `3-6` in parent-two. So, for child-two, we will have `child-two = [3 6 2 0 5 1 7 -1 -1 -1]` until we reach a cycle in parent-two (as the element `3` is already seen). If we choose the value `4` in parent-one, we get the arc `4-9` in parent-one, `9-8` in parent-two, and we are done! So, the final result for child-two will be: <br>
`child-two = [3 6 2 0 5 1 7 4 9 8]`


<figure>
    <center> <img src="./pics/Crossover/aex.png"  alt='missing' width="450"  ><center/>
<figure/>

In [30]:
import numpy as np
from copy import deepcopy
import collections

def duplicate_list(myList):
    """
    Returns True if there are no duplicate elements in the list
    """ 
    if len(myList) == len(set(myList)):
        return False
    
    return True

def alternate_edges_crossover(parent_one, parent_two, pc): 
    """
    This function takes two parents, and performs Alternate edges crossover on them. 
    parent_one: First parent
    parent_two: Second parent
    pc: The probability of crossover
    """  
 
    print("\nParents")
    print("=================================================")
    Chromosome.describe(parent_one)
    Chromosome.describe(parent_two)
    
    chrom_length = Chromosome.get_chrom_length(parent_one)
    child_one = Chromosome(genes= np.array([-1]*chrom_length),id_=0,fitness = 125.2)   
    child_two = Chromosome(genes= np.array([-1]*chrom_length),id_=1,fitness = 125.2)  

    if np.random.rand() < pc:  # if pc is greater than random number
               
        #********************************************************************** For the first child
        target_arcs = []
        seen_list = []
        pos = 0 # position of the next element
        count = 0
        alternate = 0
        while True:
            if count == chrom_length:
                break
            if alternate%2 == 0:
                target_arcs.append(parent_one.genes[pos])
                seen_list.append(parent_one.genes[pos])
                if pos+1 != chrom_length:
                    pos+=1  
                count+=1
                pos = parent_two.genes.tolist().index(parent_one.genes[pos]) 
                alternate+=1
            else:
                target_arcs.append(parent_two.genes[pos])
                seen_list.append(parent_two.genes[pos])
                if pos+1 != chrom_length:
                    pos+=1  
                count+=1
                pos = parent_one.genes.tolist().index(parent_two.genes[pos])
                alternate+=1


            if count > 1:
                n = 0
                for k in range(len(seen_list) - 1):
                    n+=1
                    if target_arcs[-1] == seen_list[k]: 
                        break

            not_seen_list = [] # elements that are not already transfered to the child
            if count > 1 and n != len(seen_list) - 1: # if a cycle is found
                target_arcs.pop()
                for j in range(chrom_length):
                    if parent_one.genes[j] not in seen_list:
                        not_seen_list.append(parent_one.genes[j])
                if not_seen_list == []:
                    break

                if alternate%2 != 0: # it means that we reached a cycle in parent_two
                    element = not_seen_list[np.random.randint(0, len(not_seen_list))]
                    target_arcs.append(element)
                    seen_list.append(element)
                    pos = parent_one.genes.tolist().index(element)
                    if pos+1 != chrom_length:
                        pos+=1                  
                    pos = parent_two.genes.tolist().index(parent_one.genes[pos]) 

                    
                else: # we reached a cycle in parent_one
                    element = not_seen_list[np.random.randint(0, len(not_seen_list))]
                    target_arcs.append(element) 
                    seen_list.append(element)
                    pos = parent_two.genes.tolist().index(element) 
                    if pos+1 != chrom_length:
                        pos+=1  
                    pos = parent_one.genes.tolist().index(parent_two.genes[pos]) 
                
        # For assurance that child_one is an authentic answer
        #$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$ 
        duplicated_elements = [item for item, count in collections.Counter(target_arcs).items() if count > 1]
        if duplicate_list(target_arcs):
            not_seen_list = []
            for i in range(chrom_length): #filling the not_seen_list 
                if parent_one.genes[i] not in target_arcs:
                    not_seen_list.append(parent_one.genes[i])
            # replacing the duplicated elements with the elements that are not seen
            for i in range(len(duplicated_elements)): 
                location = len(target_arcs) - 1 - target_arcs[::-1].index(duplicated_elements[i]) 
                target_arcs[location] = not_seen_list.pop() # last duplicated element
        #$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
        child_one.genes = np.array(target_arcs)

        # target_arcs = list(dict.fromkeys(target_arcs))
        # seen_list = list(dict.fromkeys(seen_list))
        #************************************************************************************ For the second child
        target_arcs = []
        seen_list = []
        pos = 0 # position of the next element
        count = 0
        alternate = 1
        while True:
            if count == chrom_length:
                break
            if alternate%2 == 0:
                target_arcs.append(parent_one.genes[pos])
                seen_list.append(parent_one.genes[pos])
                if pos+1 != chrom_length:
                    pos+=1  
                count+=1
                pos = parent_two.genes.tolist().index(parent_one.genes[pos]) 
                alternate+=1
            else:
                target_arcs.append(parent_two.genes[pos])
                seen_list.append(parent_two.genes[pos])
                if pos+1 != chrom_length:
                    pos+=1  
                count+=1
                pos = parent_one.genes.tolist().index(parent_two.genes[pos])
                alternate+=1

            if count > 1:
                n = 0
                for k in range(len(seen_list) - 1):
                    n+=1
                    if target_arcs[-1] == seen_list[k]: 
                        break

            not_seen_list = [] # elements that are not already transfered to child
            if count > 1 and n != len(seen_list) - 1: # if a cycle is found
                target_arcs.pop()
                for j in range(chrom_length):
                    if parent_one.genes[j] not in seen_list:
                        not_seen_list.append(parent_one.genes[j])
                if not_seen_list == []:
                    break

                if alternate%2 != 0: #it means that we reached a cycle in parent_two
                    element = not_seen_list[np.random.randint(0, len(not_seen_list))]
                    target_arcs.append(element)
                    seen_list.append(element)
                    pos = parent_one.genes.tolist().index(element)
                    if pos+1 != chrom_length:
                        pos+=1                  
                    pos = parent_two.genes.tolist().index(parent_one.genes[pos]) 

                    
                else: # we reached a cycle in parent_one
                    element = not_seen_list[np.random.randint(0, len(not_seen_list))]
                    target_arcs.append(element) 
                    seen_list.append(element)
                    pos = parent_two.genes.tolist().index(element) 
                    if pos+1 != chrom_length:
                        pos+=1  
                    pos = parent_one.genes.tolist().index(parent_two.genes[pos]) 
                
        # For assurance that child_two is an authentic answer
        #$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$ 
        duplicated_elements = [item for item, count in collections.Counter(target_arcs).items() if count > 1]
        if duplicate_list(target_arcs):
            not_seen_list = []
            for i in range(chrom_length): #filling the not_seen_list 
                if parent_one.genes[i] not in target_arcs:
                    not_seen_list.append(parent_one.genes[i])
            # replacing the duplicated elements with the elements that are not seen
            for i in range(len(duplicated_elements)): 
                location = len(target_arcs) - 1 - target_arcs[::-1].index(duplicated_elements[i]) 
                target_arcs[location] = not_seen_list.pop() # last duplicated element
        #$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
        child_two.genes = np.array(target_arcs)
      

    else:  # if pc is less than random number then don't make any change
        child_one = deepcopy(parent_one)
        child_two = deepcopy(parent_two)

    # updating fitnesses for children
    child_one.fitness = fitness_function(child_one.genes)
    child_two.fitness = fitness_function(child_two.genes)

    return child_one, child_two


chrom0 = np.array([5, 1, 7, 8, 4, 9, 6, 2, 3, 0])
chrom1 = np.array([3, 6, 2, 0, 5, 1, 9, 8, 4, 7])
parent_one = Chromosome(chrom0, id_= 0, fitness = fitness_function(chrom0))  
parent_two = Chromosome(chrom1, id_= 1, fitness = fitness_function(chrom1))
CROSS = alternate_edges_crossover(parent_one, parent_two, 1)

if __name__ == '__main__':
    print("\nChildren")
    print("=================================================")
    for index, _ in enumerate(CROSS):  #two children
        Chromosome.describe(CROSS[index])




Parents
ID=#0, Fitness=5, 
Genes=
[5 1 7 8 4 9 6 2 3 0]
ID=#1, Fitness=5, 
Genes=
[3 6 2 0 5 1 9 8 4 7]

Children
ID=#0, Fitness=5, 
Genes=
[5 1 9 6 2 3 0 7 8 4]
ID=#1, Fitness=5, 
Genes=
[3 6 2 0 8 5 1 9 4 7]


## <a id='half-uniform'>Half Uniform Crossover (HUX)</a>

Half uniform crossover (HUX), is an extension of the uniform crossover (UX). In this crossover, we first copy the genes of parent-one into child-one and the genes of parent-two into child-two. Then, we loop over child-one and child-two and wherever the value of the gene `i` for child-one is not equal to the value of the gene `i` for child-two, we produce a uniform random real number `w` in the interval `[0, 1]` and do the following for that gene: <br> 
If `w` is less than or equal to 0.5, then we swap the value of the gene in the two children. Otherwise, no change will be made to the value of that gene in the children.

<figure>
    <center> <img src="./pics/Crossover/hux.png"  alt='missing' width="450"  ><center/>
<figure/>

In [31]:
import numpy as np
from copy import deepcopy

def half_uniform_crossover(parent_one, parent_two, pc):
    """
    parent_one: First parent
    parent_two: Second parent
    pc: The probability of crossover
    """  
    print("\nParents")
    print("=================================================")
    Chromosome.describe(parent_one)
    Chromosome.describe(parent_two)
    
    chrom_length = Chromosome.get_chrom_length(parent_one)
    child_one = Chromosome(genes= np.array([-1]*chrom_length),id_=0,fitness = 125.2)   
    child_two = Chromosome(genes= np.array([-1]*chrom_length),id_=1,fitness = 125.2)

    if np.random.rand() < pc:  # if pc is greater than random number
        # copying parents to their children
        child_one.genes = parent_one.genes
        child_two.genes = parent_two.genes

        for i in range(chrom_length):
            if child_one.genes[i] != child_two.genes[i]:
                w = np.random.uniform(low=0, high=1)
                if w <= 0.5: # swapping bits
                    child_one.genes[i], child_two.genes[i] = child_two.genes[i], child_one.genes[i]  

    else:  # if pc is less than random number then don't make any change
        child_one = deepcopy(parent_one)
        child_two = deepcopy(parent_two)
    
    # updating fitnesses for children
    child_one.fitness = fitness_function(child_one.genes)
    child_two.fitness = fitness_function(child_two.genes)

    return child_one, child_two


chrom0 = np.array([0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1])
chrom1 = np.array([1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1])
parent_one = Chromosome(chrom0, id_= 0, fitness = fitness_function(chrom0))  
parent_two = Chromosome(chrom1, id_= 1, fitness = fitness_function(chrom1))
CROSS = half_uniform_crossover(parent_one, parent_two, 1)

if __name__ == '__main__':
    print("\nChildren")
    print("=================================================")
    for index, _ in enumerate(CROSS):  # two children
        Chromosome.describe(CROSS[index])



Parents
ID=#0, Fitness=6, 
Genes=
[0 1 0 0 1 1 0 1 1 0 1]
ID=#1, Fitness=7, 
Genes=
[1 0 1 1 0 1 0 1 1 0 1]

Children
ID=#0, Fitness=6, 
Genes=
[0 0 0 1 1 1 0 1 1 0 1]
ID=#1, Fitness=7, 
Genes=
[1 1 1 0 0 1 0 1 1 0 1]


## <a id='highly-disruptive'>Highly Disruptive Crossover (HDX)</a>

Highly Disruptive Crossover is a type of crossover operator that is designed to create a high degree of diversity in the offspring population. It is achieved by randomly selecting a subset of genes from each parent and swapping them between the parents. 

Here's an example of Highly Disruptive Crossover:

Let's say we have two parent chromosomes: `10101010` and `11001100`. We randomly select a subset of genes from each parent, say the first two genes from the first parent and the last two genes from the second parent. The resulting offspring will be `00101010` and `11001110`. As you can see, the offspring has a combination of genes from both parents, but the order of the genes has been disrupted. This helps to create a high degree of diversity in the offspring population.

In Highly Disruptive Crossover, the subset of genes from each parent that are swapped do not need to be adjacent to each other. They can be separated by any number of genes. The only requirement is that the subset of genes selected from each parent should be of the same length.




<figure>
    <center> <img src="./pics/Crossover/hdx.png"  alt='missing' width="450"  ><center/>
<figure/>

In [45]:
import numpy as np
from copy import deepcopy

def highly_disruptive_crossover(parent_one, parent_two, pc):
    """
    parent_one: First parent
    parent_two: Second parent
    pc: The probability of crossover
    """  
    print("\nParents")
    print("=================================================")
    Chromosome.describe(parent_one)
    Chromosome.describe(parent_two)
    
    chrom_length = Chromosome.get_chrom_length(parent_one)
    child_one = Chromosome(genes= np.array([-1]*chrom_length),id_=0,fitness = 125.2)   
    child_two = Chromosome(genes= np.array([-1]*chrom_length),id_=1,fitness = 125.2)

    if np.random.rand() < pc:  # if pc is greater than random number
        # copying parents to their children
        child_one.genes = parent_one.genes
        child_two.genes = parent_two.genes

        # Defining the arrays (children genes)
        arr1 = child_one.genes
        arr2 = child_two.genes

        # Number of genes to swap
        swap_num = np.random.randint(1, chrom_length)

        # Generate random indices to swap for each array
        indices1 = np.random.choice(len(arr1), size=swap_num, replace=False)
        indices2 = np.random.choice(len(arr2), size=swap_num, replace=False)

        # Swap the values at the selected indices for each array
        arr1[indices1], arr2[indices2] = arr2[indices2], arr1[indices1]
 

    else:  # if pc is less than random number then don't make any change
        child_one = deepcopy(parent_one)
        child_two = deepcopy(parent_two)
    
    # updating fitnesses for children
    child_one.fitness = fitness_function(child_one.genes)
    child_two.fitness = fitness_function(child_two.genes)

    return child_one, child_two


chrom0 = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
chrom1 = np.array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
parent_one = Chromosome(chrom0, id_= 0, fitness = fitness_function(chrom0))  
parent_two = Chromosome(chrom1, id_= 1, fitness = fitness_function(chrom1))
CROSS = highly_disruptive_crossover(parent_one, parent_two, 1)

if __name__ == '__main__':
    print("\nChildren")
    print("=================================================")
    for index, _ in enumerate(CROSS):  # two children
        Chromosome.describe(CROSS[index])



Parents
ID=#0, Fitness=0, 
Genes=
[0 0 0 0 0 0 0 0 0 0 0]
ID=#1, Fitness=11, 
Genes=
[1 1 1 1 1 1 1 1 1 1 1]

Children
ID=#0, Fitness=6, 
Genes=
[1 0 0 1 1 0 1 0 1 1 0]
ID=#1, Fitness=5, 
Genes=
[0 1 0 1 1 1 1 0 0 0 0]


## <a id='blend'>Blend Crossover (BLX)</a>

To perform blend crossover, we need three parameters. 
- A list named `d` which will calculate the distance between the gene values of the parents. 
- A positive real parameter named `a` in the interval [0, 1] which is selected randomly before the operation. 
- Then for each gene of the children, we create a random real uniform number `u` for each offspring. The value of the gene in that position will become this random real uniform number. `u` is calculated as the following:

> u ~ ( min(parent_one[i], parent_two[i]) - (a * d[i]), max(parent_one[i], parent_two[i]) + (a * d[i]) ).

<figure>
    <center> <img src="./pics/Crossover/blend_crossover.png"  alt='missing' width="450"  ><center/>
<figure/>

In [33]:
import numpy as np
from copy import deepcopy 

def blend_crossover(parent_one, parent_two, pc):
    """
    This function takes two parents, and performs Blend crossover on them. 
    parent_one: First parent
    parent_two: Second parent
    pc: The probability of crossover
    """  

    print("\nParents")
    print("=================================================")
    Chromosome.describe(parent_one)
    Chromosome.describe(parent_two)
    
    chrom_length = Chromosome.get_chrom_length(parent_one)
    child_one = Chromosome(genes= np.array([0.0]*chrom_length),id_=0,fitness = 125.2)   
    child_two = Chromosome(genes= np.array([0.0]*chrom_length),id_=1,fitness = 125.2)
  

    if np.random.rand() < pc:  # if pc is greater than random number
        distance_vector = []
        for i in range(chrom_length):
            distance_vector.append(abs(parent_one.genes[i] - parent_two.genes[i]))
        # print(distance_vector)
        a = np.random.uniform(low = 0, high = 1)
        for i in range(chrom_length):
            w = np.random.uniform(low = abs(np.minimum(parent_one.genes[i], parent_two.genes[i]) - a * distance_vector[i]), high = abs(np.maximum(parent_one.genes[i], parent_two.genes[i]) + a * distance_vector[i]))
            child_one.genes[i] = round(w, 3) 
            w = np.random.uniform(low = abs(np.minimum(parent_one.genes[i], parent_two.genes[i]) - a * distance_vector[i]), high = abs(np.maximum(parent_one.genes[i], parent_two.genes[i]) + a * distance_vector[i]))
            child_two.genes[i] = round(w, 3)

    else:  # if pc is less than random number then don't make any change
        child_one = deepcopy(parent_one)
        child_two = deepcopy(parent_two)

    # updating fitnesses for children
    child_one.fitness = fitness_function(child_one.genes)
    child_two.fitness = fitness_function(child_two.genes)

    return child_one, child_two



chrom0 = np.array([0.121, 0.152, 0.231, 0.143, 0.732, 0.315, 0.434, 0.633])
chrom1 = np.array([0.765, 0.168, 0.914, 0.435, 0.893, 0.332, 0.981, 0.194])
parent_one = Chromosome(chrom0, id_= 0, fitness = fitness_function(chrom0))  
parent_two = Chromosome(chrom1, id_= 1, fitness = fitness_function(chrom1))
CROSS = blend_crossover(parent_one, parent_two, 1)

if __name__ == '__main__':
    print("\nChildren")
    print("=================================================")
    for index, _ in enumerate(CROSS):  #two children
        Chromosome.describe(CROSS[index])




Parents
ID=#0, Fitness=6, 
Genes=
[0.121 0.152 0.231 0.143 0.732 0.315 0.434 0.633]
ID=#1, Fitness=4, 
Genes=
[0.765 0.168 0.914 0.435 0.893 0.332 0.981 0.194]

Children
ID=#0, Fitness=5, 
Genes=
[0.125 0.151 0.701 0.093 0.852 0.32  0.909 0.202]
ID=#1, Fitness=7, 
Genes=
[0.334 0.165 0.347 0.255 0.755 0.33  0.303 0.18 ]


## <a id='linear-recombination'>Linear Recombination Crossover</a>

In linear recombination crossover, we select two parents and create three offspring. If we name the first parent as parent-one and the second parent as parent-two, the three offspring are calculated as follow: <br>
<span style="color:#3578EE">**First offspring:**</span> `parent_one + parent_two` <br>
<span style="color:#EE6897">**Second offspring:**</span> `(1.5)*parent_one - (0.5)*parent_two` <br>
<span style="color:#25EE97">**Third offspring:**</span> `(-0.5)*parent_one + (1.5)*parent_two` <br>

After the production of the offspring, we choose the two fittest offspring and return them as the result.

<figure>
    <center> <img src="./pics/Crossover/linear_recombination_crossover.png"  alt='missing' width="450"  ><center/>
<figure/>

In [34]:
import numpy as np
from copy import deepcopy 

def linear_recombination_crossover(parent_one, parent_two, pc): 
    """
    This function takes two parents, and performs Linear crossover on them. 
    parent_one: First parent
    parent_two: Second parent
    pc: The probability of crossover
    """  

    print("\nParents")
    print("=================================================")
    Chromosome.describe(parent_one)
    Chromosome.describe(parent_two)
    
    chrom_length = Chromosome.get_chrom_length(parent_one) 
    child_one = Chromosome(genes= np.array([0.0]*chrom_length),id_=0,fitness = 130.3)   
    child_two = Chromosome(genes= np.array([0.0]*chrom_length),id_=1,fitness = 129.2)
    child_three = Chromosome(genes= np.array([0.0]*chrom_length),id_=2,fitness = 129.3)

    if np.random.rand() < pc:  # if pc is greater than random number

        child_one.genes = abs(parent_one.genes + parent_two.genes)
        child_two.genes = abs((1.5)*parent_one.genes - (0.5)*parent_two.genes)
        child_three.genes = abs((-0.5)*parent_one.genes + (1.5)*parent_two.genes)


        children = [] # List containing the children
        children.append(child_one)
        children.append(child_two)
        children.append(child_three)

        fitnesses = [] # List containing the fitnesses
        fitnesses.append(fitness_function(child_one.genes))
        fitnesses.append(fitness_function(child_two.genes))
        fitnesses.append(fitness_function(child_three.genes))

        # removing the child with the worth fitness
        minimum_fitness_index = fitnesses.index(np.min(fitnesses))
        children.pop(minimum_fitness_index) 


        child_one = children[0]
        child_two = children[1]

    else:  # if pc is less than random number then don't make any change
        child_one = deepcopy(parent_one)
        child_two = deepcopy(parent_two)
    
    # updating fitnesses for children
    child_one.fitness = fitness_function(child_one.genes)
    child_two.fitness = fitness_function(child_two.genes)

    return child_one, child_two


chrom0 = np.array([0.121, 0.152, 0.231, 0.143, 0.732, 0.315, 0.434, 0.633])
chrom1 = np.array([0.765, 0.168, 0.914, 0.435, 0.893, 0.332, 0.981, 0.194])
parent_one = Chromosome(chrom0, id_= 0, fitness = fitness_function(chrom0))  
parent_two = Chromosome(chrom1, id_= 1, fitness = fitness_function(chrom1))
CROSS = linear_recombination_crossover(parent_one, parent_two, 1)

if __name__ == '__main__':
    print("\nChildren")
    print("=================================================")
    for index, _ in enumerate(CROSS):  #two children
        Chromosome.describe(CROSS[index])




Parents
ID=#0, Fitness=6, 
Genes=
[0.121 0.152 0.231 0.143 0.732 0.315 0.434 0.633]
ID=#1, Fitness=4, 
Genes=
[0.765 0.168 0.914 0.435 0.893 0.332 0.981 0.194]

Children
ID=#1, Fitness=6, 
Genes=
[0.201  0.144  0.1105 0.003  0.6515 0.3065 0.1605 0.8525]
ID=#2, Fitness=3, 
Genes=
[1.087  0.176  1.2555 0.581  0.9735 0.3405 1.2545 0.0255]


## <a id='directional-heuristic'>Directional Heuristic Crossover</a>

In directional heuristic crossover, for gene `i` of the child, we have: <br>
`child-one[i] = U(0, 1)*(parent-two[i] - parent-one[i]) + parent-two[i]` <br>
where U(0, 1) is a random real number between 0 and 1.


<figure>
    <center> <img src="./pics/Crossover/dhx.png"  alt='missing' width="600"  ><center/>
<figure/>

In [35]:
import numpy as np
from copy import deepcopy

def directional_heuristic_crossover(parent_one, parent_two, pc):
    """
    This function takes two parents, and performs Directional heuristic crossover on them.
    This crossover is subject to the constraint that the parent_two cannot be worth than parent_one. 
    parent_one: First parent
    parent_two: Second parent
    pc: The probability of crossover
    """  

    # if parent_one is better, switch the parents
    if parent_one.fitness > parent_two.fitness: 
        parent_one, parent_two = parent_two, parent_one
    
    print("\nParents")
    print("=================================================")
    Chromosome.describe(parent_one)
    Chromosome.describe(parent_two)
    
    chrom_length = Chromosome.get_chrom_length(parent_one)
    child_one = Chromosome(genes= np.array([0.0]*chrom_length), id_=0, fitness = 125.2)  # child
  

    if np.random.rand() < pc:  # if pc is greater than random number
        for i in range(chrom_length):
            child_one.genes[i] = abs(round(np.random.uniform(low=0.0, high=1.0)*(parent_two.genes[i] - parent_one.genes[i]) + parent_two.genes[i], 3))
            
        

    else:  # if pc is less than random number then don't make any change
        child_one = deepcopy(parent_one)
        

    # updating fitnesses for children
    child_one.fitness = fitness_function(child_one.genes)

    return child_one



chrom0 = np.array([0.121, 0.152, 0.231, 0.143, 0.732, 0.315, 0.434, 0.633])
chrom1 = np.array([0.765, 0.168, 0.914, 0.435, 0.893, 0.332, 0.981, 0.194])
parent_one = Chromosome(chrom0, id_= 0, fitness = fitness_function(chrom0))  
parent_two = Chromosome(chrom1, id_= 1, fitness = fitness_function(chrom1))
CROSS = directional_heuristic_crossover(parent_one, parent_two, 1)

if __name__ == '__main__':
    print("\nChildren")
    print("=================================================")

    Chromosome.describe(CROSS)



Parents
ID=#1, Fitness=4, 
Genes=
[0.765 0.168 0.914 0.435 0.893 0.332 0.981 0.194]
ID=#0, Fitness=6, 
Genes=
[0.121 0.152 0.231 0.143 0.732 0.315 0.434 0.633]

Children
ID=#0, Fitness=6, 
Genes=
[0.193 0.141 0.144 0.003 0.686 0.312 0.317 0.926]


## <a id='geometrical'>Geometrical Crossover</a>

In geometrical crossover, one offspring is produced, and the value of its genes is calculated by the geometrical average of the same genes in parents. If we have `n` parents, each gene of the offspring will be produced as follows: 

`child-one = (parent_one)^(a_1) * (parent_two)^(a_2) * … * (parent_n)^(a_n)`

where $∑_{i=1}^{n} a_i = 1$. This crossover can also be used in multi-parent recombination.

<figure>
    <center> <img src="./pics/Crossover/geometrical_crossover.png"  alt='missing' width="450"  ><center/>
<figure/>

In [36]:
import numpy as np
from copy import deepcopy
import math

def geometrical_crossover(parent_one, parent_two, pc):
    """
    This function takes two parents, and performs geometrical crossover on them.
    parent_one: First parent
    parent_two: Second parent
    pc: The probability of crossover
    """  

    print("\nParents")
    print("=================================================")
    Chromosome.describe(parent_one)
    Chromosome.describe(parent_two)
    
    chrom_length = Chromosome.get_chrom_length(parent_one)
    child_one = Chromosome(genes= np.array([0.0]*chrom_length),id_=0,fitness = 125.2)   
  
    if np.random.rand() < pc:  # if pc is greater than random number

        alpha1 = np.random.uniform(low=0.0, high=0.5)  # alpha1 + alpha2 = 1
        alpha2 = 1 - alpha1   

        for i in range(chrom_length):
            child_one.genes[i] = round((parent_one.genes[i] ** alpha1) * (parent_two.genes[i] ** alpha2), 3)

    else:  # if pc is less than random number then don't make any change
        child_one = deepcopy(parent_one)

    # updating fitnesses for children
    child_one.fitness = fitness_function(child_one.genes)
    
    return child_one



chrom0 = np.array([0.121, 0.152, 0.231, 0.143, 0.732, 0.315, 0.434, 0.633])
chrom1 = np.array([0.765, 0.168, 0.914, 0.435, 0.893, 0.332, 0.981, 0.194])
parent_one = Chromosome(chrom0, id_= 0, fitness = fitness_function(chrom0))  
parent_two = Chromosome(chrom1, id_= 1, fitness = fitness_function(chrom1))
CROSS = geometrical_crossover(parent_one, parent_two, 1)

if __name__ == '__main__':
    print("\nChildren")
    print("=================================================")
    Chromosome.describe(CROSS)




Parents
ID=#0, Fitness=6, 
Genes=
[0.121 0.152 0.231 0.143 0.732 0.315 0.434 0.633]
ID=#1, Fitness=4, 
Genes=
[0.765 0.168 0.914 0.435 0.893 0.332 0.981 0.194]

Children
ID=#0, Fitness=5, 
Genes=
[0.371 0.162 0.533 0.281 0.826 0.325 0.712 0.309]


## <a id='simulated-binary'>Simulated Binary Crossover (SBX)</a>

Simulated Binary Crossover (SBX) is a crossover operator used in genetic algorithms. It is designed to work with real-valued chromosomes. The SBX operator works by simulating the binary crossover operator used in binary-coded GAs. The idea is to retain the property of binary coding with single-point crossover, which states that the average of the parent chromosomes is equal to the average of the child chromosomes. In SBX, this property is retained by using a probability distribution that simulates the binary crossover. The crossover operator selects two parent chromosomes and generates two offspring chromosomes by applying the SBX operator. The SBX operator works by generating a random number between 0 and 1, and then using this number to calculate the beta value. The beta value is then used to calculate the offspring chromosomes from the parent chromosomes.

Here is an example of how SBX works:

Let's say we have two parent chromosomes: `p1 = [1.5, 2.0, 3.0]` and `p2 = [2.5, 3.0, 4.0]`. We randomly select a beta value between 0 and 1, say `beta = 0.5`. The resulting offspring chromosomes will be:

```
c1 = [1.75, 2.5, 3.5]
c2 = [2.25, 2.5, 3.5]
```

As you can see, the offspring chromosomes have a combination of genes from both parents, but the values have been modified based on the beta value.



<figure>
    <center> <img src="./pics/Crossover/simulated_binary_crossover.png"  alt='missing' width="450"  ><center/>
<figure/>

In [37]:
import numpy as np
from copy import deepcopy

def simulated_binary_crossover(parent_one, parent_two, pc):
    """
    This function takes two parents, and performs Simulated binary crossover on them. 
    parent_one: First parent
    parent_two: Second parent
    pc: The probability of crossover
    """  

    print("\nParents")
    print("=================================================")
    Chromosome.describe(parent_one)
    Chromosome.describe(parent_two)
    
    chrom_length = Chromosome.get_chrom_length(parent_one)
    child_one = Chromosome(genes= np.array([0.0]*chrom_length),id_=0,fitness = 125.2)   
    child_two = Chromosome(genes= np.array([0.0]*chrom_length),id_=1,fitness = 125.2)
  

    if np.random.rand() < pc:  # if pc is greater than random number      
        distribution_index = np.random.randint(1, chrom_length) #distribution_index > 0
        factor = 0
        for i in range(chrom_length):
            random_number = np.random.uniform(low=0.0, high=1.0)
            if random_number <= 0.5:
                factor = (2 * random_number) ** (1 / (distribution_index + 1))
            else:
                factor = (1 / (2 * (1 - random_number))) ** (1 / (distribution_index + 1))

            child_one.genes[i] = round(abs(0.5 * ((1 + factor) * parent_one.genes[i] + (1 - factor) * parent_two.genes[i])), 3)   
            child_two.genes[i] = round(abs(0.5 * ((1 - factor) * parent_one.genes[i] + (1 + factor) * parent_two.genes[i])), 3)  

    else:  # if pc is less than random number then don't make any change
        child_one = deepcopy(parent_one)
        child_two = deepcopy(parent_two)

    # updating fitnesses for children
    child_one.fitness = fitness_function(child_one.genes)
    child_two.fitness = fitness_function(child_two.genes)

    return child_one, child_two



chrom0 = np.array([0.121, 0.152, 0.231, 0.143, 0.732, 0.315, 0.434, 0.633])
chrom1 = np.array([0.765, 0.168, 0.914, 0.435, 0.893, 0.332, 0.981, 0.194])
parent_one = Chromosome(chrom0, id_= 0, fitness = fitness_function(chrom0))  
parent_two = Chromosome(chrom1, id_= 1, fitness = fitness_function(chrom1))
CROSS = simulated_binary_crossover(parent_one, parent_two, 1)

if __name__ == '__main__':
    print("\nChildren")
    print("=================================================")
    for index, _ in enumerate(CROSS):  #two children
        Chromosome.describe(CROSS[index])




Parents
ID=#0, Fitness=6, 
Genes=
[0.121 0.152 0.231 0.143 0.732 0.315 0.434 0.633]
ID=#1, Fitness=4, 
Genes=
[0.765 0.168 0.914 0.435 0.893 0.332 0.981 0.194]

Children
ID=#0, Fitness=6, 
Genes=
[0.14  0.15  0.201 0.143 0.731 0.316 0.371 0.637]
ID=#1, Fitness=4, 
Genes=
[0.746 0.17  0.944 0.435 0.894 0.331 1.044 0.19 ]


## <a id='hill-climbing'>Hill-climbing Crossover</a>

Hill-climbing is a local search algorithm that starts with an initial solution and iteratively improves the solution by making small modifications to it. Hill-climbing crossover is a variation of hill-climbing that is used in genetic algorithms. In hill-climbing crossover, the algorithm starts with an initial population of solutions, and then selects pairs of solutions to be crossed over. In hill-climbing crossover, we use uniform crossover to produce offspring; but this time, we keep doing it until an offspring is produced which has a better fitness than one of her parents.


<figure>
    <center> <img src="./pics/Crossover/hill_cross.png"  alt='missing' width="600"  ><center/>
<figure/>

In [38]:
import numpy as np
from copy import deepcopy

def claculate_mask(chrom_size, px):
    """
    This method, performs the mask calculation for the Uniform crossover.
    chrom_size: The mask has the same size of the chromosome.
    px: A control parameter for the number of ones in the mask vector.
    """  
    mask = [0]*chrom_size
    for i in range(chrom_size):
        prob = np.random.rand()
        if prob <= px:
            mask[i]=1
    return mask
            

def uniform_crossover(parent_one, parent_two, pc):
    """
    This function takes two parents, and performs Uniform crossover on them. 
    parent_one: First parent
    parent_two: Second parent
    pc: The probability of crossover
    """  
    
    chrom_length = Chromosome.get_chrom_length(parent_one)
    child_one = Chromosome(genes= np.array([0]*chrom_length),id_=0,fitness = 125.2)   
    child_two = Chromosome(genes= np.array([0]*chrom_length),id_=1,fitness = 125.2)
    if np.random.rand() < pc:  # if pc is greater than random number
        mask = claculate_mask(chrom_length,np.random.uniform(low=0.2, high=0.85))
        for i in range(chrom_length):
            if mask[i]==1:
                child_one.genes[i] = parent_one.genes[i]
                child_two.genes[i] = parent_two.genes[i]
            else:
                child_one.genes[i] = parent_two.genes[i]
                child_two.genes[i] = parent_one.genes[i]
           
    else:  # if pc is less than random number then don't make any change
        child_one = deepcopy(parent_one)
        child_two = deepcopy(parent_two)
    return child_one, child_two


def hillclimbing_crossover(parent_one, parent_two, pc):
    """
    parent_one: First parent
    parent_two: Second parent
    pc: The probability of crossover
    """  

    parent_one = Chromosome(genes= np.array([0, 1, 0, 0, 1, 0, 0, 1, 1, 1]),id_=0,fitness = 5)   
    parent_two = Chromosome(genes= np.array([1, 0, 1, 0, 0, 1, 0, 1, 0, 0]),id_=1,fitness = 4) #repair fitness_function
    
    print("\nParents")
    print("=================================================")
    Chromosome.describe(parent_one)
    Chromosome.describe(parent_two)
    
    chrom_length = Chromosome.get_chrom_length(parent_one)
    child_one = Chromosome(genes= np.array([0]*chrom_length),id_=0,fitness = 125.2)   
    child_two = Chromosome(genes= np.array([0]*chrom_length),id_=1,fitness = 125.2)
  

    if np.random.rand() < pc:  # if pc is greater than random number
        maximum_parent_fitness = np.maximum(parent_one.fitness, parent_two.fitness)
        child_one, child_two = uniform_crossover(parent_one, parent_two, 1)
        child_one.fitness = fitness_function(child_one.genes)
        child_two.fitness = fitness_function(child_two.genes)
        maximum_child_fitness = np.maximum(child_one.fitness, child_two.fitness)
        
        # Keep going until one of the children becomes better than parents
        if maximum_child_fitness <= maximum_parent_fitness:
            while maximum_child_fitness <= maximum_parent_fitness: 
                child_one, child_two = uniform_crossover(child_one, child_two, 1)
                child_one.fitness = fitness_function(child_one.genes)
                child_two.fitness = fitness_function(child_two.genes)
                maximum_child_fitness = np.maximum(child_one.fitness, child_two.fitness)


    else:  # if pc is less than random number then don't make any change
        child_one = deepcopy(parent_one)
        child_two = deepcopy(parent_two)
    
    # updating fitnesses for children
    child_one.fitness = fitness_function(child_one.genes)
    child_two.fitness = fitness_function(child_two.genes)

    return child_one, child_two



chrom0 = np.array([0, 1, 0, 0, 1, 0, 0, 1, 1, 1])
chrom1 = np.array([1, 0, 1, 0, 0, 1, 0, 1, 0, 0])
parent_one = Chromosome(chrom0, id_= 0, fitness = fitness_function(chrom0))  
parent_two = Chromosome(chrom1, id_= 1, fitness = fitness_function(chrom1))
CROSS = hillclimbing_crossover(parent_one, parent_two, 1)

if __name__ == '__main__':
    print("\nChildren")
    print("=================================================")
    for index, _ in enumerate(CROSS):  #two children
        Chromosome.describe(CROSS[index])



Parents
ID=#0, Fitness=5, 
Genes=
[0 1 0 0 1 0 0 1 1 1]
ID=#1, Fitness=4, 
Genes=
[1 0 1 0 0 1 0 1 0 0]

Children
ID=#0, Fitness=3, 
Genes=
[0 1 0 0 0 0 0 1 1 0]
ID=#1, Fitness=6, 
Genes=
[1 0 1 0 1 1 0 1 0 1]


## <a id='arithmetic'>Arithmetic Crossover</a>

Arithmetic Crossover is a type of crossover operator used in genetic algorithms. It is designed to work with real-valued chromosomes. The arithmetic crossover operator works by generating a new offspring chromosome by taking a weighted average of the corresponding genes in the parent chromosomes. The weight is determined by a random number between 0 and 1, and the offspring chromosome is generated as follows:

```
c[i] = alpha * p1[i] + (1 - alpha) * p2[i]
```

where `c[i]` is the `i`-th gene in the offspring chromosome, `p1[i]` and `p2[i]` are the corresponding genes in the parent chromosomes, and `alpha` is the weight factor. The value of `alpha` is usually chosen randomly from a uniform distribution between 0 and 1.

Here's an example of how arithmetic crossover works:

Let's say we have two parent chromosomes: `p1 = [1.5, 2.0, 3.0]` and `p2 = [2.5, 3.0, 4.0]`. We randomly select a weight factor `alpha = 0.5`. The resulting offspring chromosome will be:

```
c = [2.0, 2.5, 3.5]
```

As you can see, the offspring chromosome has a combination of genes from both parents, but the values have been modified based on the weight factor.


<figure>
    <center> <img src="./pics/Crossover/arithmetic_crossover.png"  alt='missing' width="450"  ><center/>
<figure/>

In [39]:
import numpy as np
from copy import deepcopy

def arithmetic_crossover(parent_one, parent_two, pc):
    """
    parent_one: First parent
    parent_two: Second parent
    pc: The probability of crossover
    """  

    print("\nParents")
    print("=================================================")
    Chromosome.describe(parent_one)
    Chromosome.describe(parent_two)
    
    chrom_length = Chromosome.get_chrom_length(parent_one)
    child_one = Chromosome(genes= np.array([0.0]*chrom_length),id_=0,fitness = 125.2)   
    child_two = Chromosome(genes= np.array([0.0]*chrom_length),id_=1,fitness = 125.2)
  

    if np.random.rand() < pc:  # if pc is greater than random number  
        alpha = np.random.uniform(low=0.0, high=1.0)
        for i in range(chrom_length):
            child_one.genes[i] = round((1 - alpha) * parent_one.genes[i] + alpha * parent_two.genes[i], 3)
            child_two.genes[i] = round(alpha * parent_one.genes[i] + (1 - alpha) * parent_two.genes[i], 3)    


    else:  # if pc is less than random number then don't make any change
        child_one = deepcopy(parent_one)
        child_two = deepcopy(parent_two)
    
    # updating fitnesses for children
    child_one.fitness = fitness_function(child_one.genes)
    child_two.fitness = fitness_function(child_two.genes)

    return child_one, child_two


chrom0 = np.array([0.121, 0.152, 0.231, 0.143, 0.732, 0.315, 0.434, 0.633])
chrom1 = np.array([0.765, 0.168, 0.914, 0.435, 0.893, 0.332, 0.981, 0.194])
parent_one = Chromosome(chrom0, id_= 0, fitness = fitness_function(chrom0))  
parent_two = Chromosome(chrom1, id_= 1, fitness = fitness_function(chrom1))
CROSS = arithmetic_crossover(parent_one, parent_two, 1)

if __name__ == '__main__':
    print("\nChildren")
    print("=================================================")
    for index, _ in enumerate(CROSS):  #two children
        Chromosome.describe(CROSS[index])



Parents
ID=#0, Fitness=6, 
Genes=
[0.121 0.152 0.231 0.143 0.732 0.315 0.434 0.633]
ID=#1, Fitness=4, 
Genes=
[0.765 0.168 0.914 0.435 0.893 0.332 0.981 0.194]

Children
ID=#0, Fitness=4, 
Genes=
[0.592 0.164 0.731 0.357 0.85  0.327 0.834 0.312]
ID=#1, Fitness=5, 
Genes=
[0.294 0.156 0.414 0.221 0.775 0.32  0.581 0.515]


## References 

1. Engelbrecht, A. P. (2007). Computational intelligence: An introduction (2nd ed.). John Wiley & Sons.

2. Crossover (genetic algorithm). (n.d.). In Wikipedia. Retrieved December 27, 2023, from https://en.wikipedia.org/wiki/Crossover_%28genetic_algorithm%29

3. Genetic Algorithm: A Learning Experience. (n.d.). In UNSW Sites. Retrieved December 27, 2023, from https://www.cse.unsw.edu.au/~cs9417ml/GA2/crossover_recom.html

4. Crossover, edge recombination operator, and fly algorithm. (n.d.). In INDIAai. Retrieved December 27, 2023, from https://indiaai.gov.in/article/crossover-edge-recombination-operator-and-fly-algorithm

5. Crossover in Genetic Algorithm. (n.d.). In GeeksforGeeks. Retrieved December 27, 2023, from https://www.geeksforgeeks.org/crossover-in-genetic-algorithm/

6. Easiest way to implement cycle crossover. (n.d.). In Code Review Stack Exchange. Retrieved December 27, 2023, from https://codereview.stackexchange.com/questions/226179/easiest-way-to-implement-cycle-crossover

7. Partially Mapped Crossover in Genetic Algorithms. (n.d.). In Baeldung. Retrieved December 27, 2023, from https://www.baeldung.com/cs/ga-pmx-operator

8. pmx: Partially Mapped Crossover. (n.d.). In R Package Documentation. Retrieved December 27, 2023, from https://rdrr.io/cran/adana/man/pmx.html

9. Bing, M. (2023). Recombination in genetic algorithms [Graphic art]. Image Creator from Microsoft Designer.

10. cpc: Count-preserving Crossover (CPC). (n.d.). In R Package Documentation. Retrieved December 27, 2023, from https://rdrr.io/cran/adana/man/cpc.html.

11. Gwiazda, T. (n.d.). Count-preserving Crossover. Retrieved December 27, 2023, from http://www.tomaszgwiazda.com/countPX.htm

12. Leps, M. (n.d.). Genetic Algorithms. Retrieved December 27, 2023, from https://mech.fsv.cvut.cz/~leps/teaching/mom/lectures/lecture06_GA.pdf

13. Crossover (genetic algorithm). (n.d.). In bionity.com. Retrieved December 27, 2023, from https://www.bionity.com/en/encyclopedia/Crossover_%28genetic_algorithm%29.html

14. Gwiazda, T. (n.d.). Introduction to: Genetic algorithms reference Volume I Crossover for .... Retrieved December 27, 2023, from http://tomaszgwiazda.com/wstep.htm

15. Kaur, H., & Singh, S. (2017). Multi-objective optimisation of reliable product-plant network .... Applied Network Science, 2(1), 1-16. https://doi.org/10.1007/s41109-017-0058-8

16. Srinivas, M., & Patnaik, L. M. (2022). A Study of Crossover Operators in Genetic Algorithms. In L. M. Patnaik, & M. Srinivas (Eds.), Handbook of Genetic Algorithms (pp. 23-45). Springer. https://doi.org/10.1007/978-981-16-3128-3_2

17. Goldberg, D. E. (2014). Genetic Algorithms. In J. Sammut, & G. I. Webb (Eds.), Encyclopedia of Machine Learning and Data Mining (pp. 487-491). Springer. https://doi.org/10.1007/978-3-319-07124-4_28

18. Srinivas, M., & Patnaik, L. M. (2022). A Study of Crossover Operators in Genetic Algorithms. In L. M. Patnaik, & M. Srinivas (Eds.), Handbook of Genetic Algorithms (pp. 23-45). Springer. https://doi.org/10.1007/978-981-16-3128-3_2

19. Aegis Software. (n.d.). The Impact of Disruptive Industrial Crossover. Retrieved December 27, 2023, from https://www.aiscorp.com/blog/blurred-and-disrupted-the-impact-of-industry-crossover/

20. Goldberg, D. E. (2014). Genetic Algorithms. In J. Sammut, & G. I. Webb (Eds.), Encyclopedia of Machine Learning and Data Mining (pp. 487-491). Springer. https://doi.org/10.1007/978-3-319-07124-4_28

21. Unequal crossing over. (n.d.). In Wikipedia. Retrieved December 27, 2023, from https://en.wikipedia.org/wiki/Unequal_crossing_over

22. University of Texas at Austin. (2018, July 9). Jumping genes: Cross species transfer of genes has driven evolution. ScienceDaily. Retrieved December 27, 2023, from https://www.sciencedaily.com/releases/2018/07/180709101216.htm

23. Srinivas, M., & Patnaik, L. M. (2022). A Study of Crossover Operators in Genetic Algorithms. In L. M. Patnaik, & M. Srinivas (Eds.), Handbook of Genetic Algorithms (pp. 23-45). Springer. https://doi.org/10.1007/978-981-16-3128-3_2

24. Székely, G. J., & Rizzo, M. L. (2021). Approximate distance correlation for selecting highly informative variables. PLoS Computational Biology, 17(11), e1009548. https://doi.org/10.1371/journal.pcbi.1009548

25. Deb, K., & Agrawal, S. (2003). Simulated Binary Crossover for Continuous Search Space. Complex Systems, 15(3), 255-282. https://doi.org/10.1142/S0129183103005783

26. Deb, K. (2001). Simulated binary crossover for continuous search space. Complex Systems, 13(6), 493-511. https://doi.org/10.1007/s00453-002-0974-z

27. Stack Overflow. (2014, March 17). Simulated Binary Crossover (SBX) crossover operator example. Retrieved December 27, 2023, from https://stackoverflow.com/questions/22457941/simulated-binary-crossover-sbx-crossover-operator-example

28. Sudhoff, S. D. (n.d.). Real-Coded Genetic Algorithms. Retrieved December 27, 2023, from https://engineering.purdue.edu/~sudhoff/ee630/Lecture04.pdf

29. Paskorn, T. (n.d.). Simulated Binary Crossover. Retrieved December 27, 2023, from https://www.slideshare.net/paskorn/simulated-binary-crossover-presentation

30. undefined. (n.d.). Retrieved December 27, 2023, from http://www.slideshare.net/

31. University of Massachusetts Boston. (n.d.). DSSG. Retrieved December 27, 2023, from http://dssg.cs.umb.edu

32. mcga. (n.d.). sbx_crossover: Performs sbx (simulated binary) crossover operation on a .... Retrieved December 27, 2023, from https://rdrr.io/cran/mcga/man/sbx_crossover.html

33. pymoo. (n.d.). Crossover. Retrieved December 27, 2023, from https://pymoo.org/operators/crossover.html

34. Kuo, T. T., & Lin, C. H. (2011). Hill-Climbing search and diversification within an evolutionary algorithm for feature selection. BioData Mining, 4(1), 23. https://doi.org/10.1186/1756-0381-4-23

35. Stack Overflow. (2016, Nov 1). How is a genetic algorithm with only selection and mutation same as a hill climb? Retrieved December 27, 2023, from https://stackoverflow.com/questions/40334711/how-is-a-genetic-algorithm-with-only-selection-and-mutation-same-as-a-hill-climb

36. Testbook. (n.d.). Genetic Algorithms MCQ [Free PDF]. Retrieved December 27, 2023, from https://testbook.com/objective-questions/mcq-on-genetic-algorithms--5eea6a0e39140f30f369e524

37. Hansen, N. (2015). An Analysis of Integration of Hill Climbing in Crossover and Mutation Operators of Evolution Strategies. In Proceedings of the 2015 Annual Conference on Genetic and Evolutionary Computation (pp. 209-216). ACM. https://doi.org/10.1145/2739480.2754773

38. Thierens, D., & Bosman, P. A. (2012). Real-Coded Memetic Algorithms with Crossover Hill-Climbing. Evolutionary Computation, 20(2), 273-300. https://doi.org/10.1162/EVCO_a_00053

39. Garg, A. (2021, August 16). Crossover Operators in Genetic Algorithm. Medium. Retrieved December 27, 2023, from https://medium.com/geekculture/crossover-operators-in-ga-cffa77cdd0c8

40. Crossover (genetic algorithm). (n.d.). In Wikipedia. Retrieved December 27, 2023, from https://en.wikipedia.org/wiki/Crossover_%28genetic_algorithm%29

41. Scassellati, B. (n.d.). Genetic Algorithms. Retrieved December 27, 2023, from https://zoo.cs.yale.edu/classes/cs470/lectures/s2019/17-Genetic-Algorithms.pdf

42. Tutorialspoint. (n.d.). Genetic Algorithms - Crossover. Retrieved December 27, 2023, from https://www.tutorialspoint.com/genetic_algorithms/genetic_algorithms_crossover.htm

